In \[1\]:

    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats
    from scipy.stats import shapiro

In \[2\]:

    %%html
    <style>
    .dataframe td {
        white-space: nowrap;
    }
    </style>

In \[3\]:

    # PATH
    PATH = 'Data/'

### Import Data & Processing<a href="#Import-Data-&amp;-Processing" class="anchor-link">¶</a>

#### Columns<a href="#Columns" class="anchor-link">¶</a>

\| Item information, Store information

In \[4\]:

    # Columns
    COLS_ITM = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    COLS_DATE = ['d_' + str(i) for i in range(1, 366)]

#### Sales Data<a href="#Sales-Data" class="anchor-link">¶</a>

\| We will take only the training data set and filter the first year

In \[5\]:

    # Import
    df = pd.read_csv(PATH + 'sales_train_evaluation.csv')
    print("{:,} records for train data set".format(len(df)))
    # Change id to match with price dataframe
    df['id'] = df['store_id'] + '-' + df['item_id']
    # Set index
    df.set_index(COLS_ITM, inplace = True)
    # Scope = Year 1
    df = df[COLS_DATE]

    df.head()

    30,490 records for train data set

Out\[5\]:

|                     |                |           |         |          |          | d_1 | d_2 | d_3 | d_4 | d_5 | d_6 | d_7 | d_8 | d_9 | d_10 | ... | d_356 | d_357 | d_358 | d_359 | d_360 | d_361 | d_362 | d_363 | d_364 | d_365 |
|---------------------|----------------|-----------|---------|----------|----------|-----|-----|-----|-----|-----|-----|-----|-----|-----|------|-----|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| id                  | item_id        | dept_id   | cat_id  | store_id | state_id |     |     |     |     |     |     |     |     |     |      |     |       |       |       |       |       |       |       |       |       |       |
| CA_1-HOBBIES_1\_001 | HOBBIES_1\_001 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0    | ... | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     |
| CA_1-HOBBIES_1\_002 | HOBBIES_1\_002 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0    | ... | 0     | 0     | 0     | 1     | 0     | 0     | 1     | 1     | 0     | 0     |
| CA_1-HOBBIES_1\_003 | HOBBIES_1\_003 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0    | ... | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     |
| CA_1-HOBBIES_1\_004 | HOBBIES_1\_004 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0    | ... | 0     | 1     | 1     | 8     | 1     | 3     | 0     | 1     | 2     | 3     |
| CA_1-HOBBIES_1\_005 | HOBBIES_1\_005 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0    | ... | 1     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     |

5 rows × 365 columns

#### Calendar<a href="#Calendar" class="anchor-link">¶</a>

\| We'll use the calendar to create a dictionnary to convert days into
weeks to get the selling prices

In \[6\]:

    # Import Calendar
    df_calendar = pd.read_csv(PATH + 'calendar.csv')
    # Date to Week
    dict_week = dict(zip(df_calendar.d.values, df_calendar.wm_yr_wk.values))
    df_calendar.head()

Out\[6\]:

|     | date       | wm_yr_wk | weekday   | wday | month | year | d   | event_name_1 | event_type_1 | event_name_2 | event_type_2 | snap_CA | snap_TX | snap_WI |
|-----|------------|----------|-----------|------|-------|------|-----|--------------|--------------|--------------|--------------|---------|---------|---------|
| 0   | 2011-01-29 | 11101    | Saturday  | 1    | 1     | 2011 | d_1 | NaN          | NaN          | NaN          | NaN          | 0       | 0       | 0       |
| 1   | 2011-01-30 | 11101    | Sunday    | 2    | 1     | 2011 | d_2 | NaN          | NaN          | NaN          | NaN          | 0       | 0       | 0       |
| 2   | 2011-01-31 | 11101    | Monday    | 3    | 1     | 2011 | d_3 | NaN          | NaN          | NaN          | NaN          | 0       | 0       | 0       |
| 3   | 2011-02-01 | 11101    | Tuesday   | 4    | 2     | 2011 | d_4 | NaN          | NaN          | NaN          | NaN          | 1       | 1       | 0       |
| 4   | 2011-02-02 | 11101    | Wednesday | 5    | 2     | 2011 | d_5 | NaN          | NaN          | NaN          | NaN          | 1       | 0       | 1       |

#### Price<a href="#Price" class="anchor-link">¶</a>

\| We'll create a pricing matrix M(n, p) with n = sku_id and p = week

In \[7\]:

    # Import
    df_price = pd.read_csv(PATH + 'sell_prices.csv')
    print("{:,} records for sales price".format(len(df_price)))
    # SKU Index
    df_price['item_store_id'] = df_price['store_id'] + '-' + df_price['item_id']
    # Pricing
    df_price = df_price.pivot(index='item_store_id', columns='wm_yr_wk', values='sell_price').fillna(0)
    # Matrix from pivot 
    matrix_price = df_price.to_numpy()
    # Dict Matrix Index
    pr_n = dict(zip(df_price.index, range(len(df_price.index))))
    pr_p = dict(zip(df_price.columns, range(len(df_price.columns))))
    print("{:,} records for sales price pivot".format(len(df_price)))
    df_price.head()

    6,841,121 records for sales price
    30,490 records for sales price pivot

Out\[7\]:

| wm_yr_wk          | 11101 | 11102 | 11103 | 11104 | 11105 | 11106 | 11107 | 11108 | 11109 | 11110 | ... | 11612 | 11613 | 11614 | 11615 | 11616 | 11617 | 11618 | 11619 | 11620 | 11621 |
|-------------------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-----|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| item_store_id     |       |       |       |       |       |       |       |       |       |       |     |       |       |       |       |       |       |       |       |       |       |
| CA_1-FOODS_1\_001 | 2.00  | 2.00  | 2.00  | 2.00  | 2.00  | 2.00  | 2.00  | 2.00  | 2.00  | 2.00  | ... | 2.24  | 2.24  | 2.24  | 2.24  | 2.24  | 2.24  | 2.24  | 2.24  | 2.24  | 2.24  |
| CA_1-FOODS_1\_002 | 7.88  | 7.88  | 7.88  | 7.88  | 7.88  | 7.88  | 7.88  | 7.88  | 7.88  | 7.88  | ... | 9.48  | 9.48  | 9.48  | 9.48  | 9.48  | 9.48  | 9.48  | 9.48  | 9.48  | 9.48  |
| CA_1-FOODS_1\_003 | 2.88  | 2.88  | 2.88  | 2.88  | 2.88  | 2.88  | 2.88  | 2.88  | 2.88  | 2.88  | ... | 3.23  | 3.23  | 3.23  | 3.23  | 3.23  | 3.23  | 3.23  | 3.23  | 3.23  | 3.23  |
| CA_1-FOODS_1\_004 | 0.00  | 0.00  | 0.00  | 0.00  | 0.00  | 0.00  | 0.00  | 0.00  | 0.00  | 0.00  | ... | 1.96  | 1.96  | 1.96  | 1.96  | 1.96  | 1.96  | 1.96  | 1.96  | 1.96  | 1.96  |
| CA_1-FOODS_1\_005 | 2.94  | 2.94  | 2.94  | 2.94  | 2.94  | 2.94  | 2.94  | 2.94  | 2.94  | 2.94  | ... | 3.54  | 3.54  | 3.54  | 3.54  | 3.54  | 3.54  | 3.54  | 3.54  | 3.54  | 3.54  |

5 rows × 282 columns

------------------------------------------------------------------------

### Statistical Analysis<a href="#Statistical-Analysis" class="anchor-link">¶</a>

#### Calculate sigma, mu<a href="#Calculate-sigma,-mu" class="anchor-link">¶</a>

In \[8\]:

    # Mean
    df['mean'] = df[COLS_DATE].mean(axis = 1)
    # Standard
    df['std'] = df[COLS_DATE].std(axis = 1)

    # Remove items not sold during the first year
    print("{:,} records for the full scope".format(len(df)))
    df = df[df['mean']>0]
    print("{:,} records for after filter".format(len(df)))
    df.reset_index(inplace = True)
    # Sigma
    df.head()

    30,490 records for the full scope
    17,055 records for after filter

Out\[8\]:

|     | id                  | item_id        | dept_id   | cat_id  | store_id | state_id | d_1 | d_2 | d_3 | d_4 | ... | d_358 | d_359 | d_360 | d_361 | d_362 | d_363 | d_364 | d_365 | mean     | std      |
|-----|---------------------|----------------|-----------|---------|----------|----------|-----|-----|-----|-----|-----|-------|-------|-------|-------|-------|-------|-------|-------|----------|----------|
| 0   | CA_1-HOBBIES_1\_002 | HOBBIES_1\_002 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 0     | 1     | 0     | 0     | 1     | 1     | 0     | 0     | 0.175342 | 0.459270 |
| 1   | CA_1-HOBBIES_1\_004 | HOBBIES_1\_004 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 1     | 8     | 1     | 3     | 0     | 1     | 2     | 3     | 1.282192 | 1.504512 |
| 2   | CA_1-HOBBIES_1\_005 | HOBBIES_1\_005 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0     | 0.794521 | 1.543854 |
| 3   | CA_1-HOBBIES_1\_008 | HOBBIES_1\_008 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 12  | 15  | 0   | 0   | ... | 20    | 26    | 0     | 9     | 14    | 0     | 8     | 18    | 6.854795 | 8.556423 |
| 4   | CA_1-HOBBIES_1\_009 | HOBBIES_1\_009 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 2   | 0   | 7   | 3   | ... | 1     | 0     | 0     | 1     | 0     | 0     | 3     | 0     | 1.816438 | 2.486613 |

5 rows × 373 columns

### Turnover, Sales Units<a href="#Turnover,-Sales-Units" class="anchor-link">¶</a>

#### Turnover by Item,Store<a href="#Turnover-by-Item,Store" class="anchor-link">¶</a>

\| Turnover = Price x Volume

In \[9\]:

    # Total Units
    df['units'] = df[COLS_DATE].sum(axis = 1)
    # Turnover = Units x Price
    df['TO'] = 0 
    for col in COLS_DATE:
        df['TO'] = df['TO'] + df[col] * df[['id', col]].apply(
            lambda t: matrix_price[pr_n[t['id']], pr_p[dict_week[col]]], axis = 1)
    # Sigma
    df.head()

Out\[9\]:

|     | id                  | item_id        | dept_id   | cat_id  | store_id | state_id | d_1 | d_2 | d_3 | d_4 | ... | d_360 | d_361 | d_362 | d_363 | d_364 | d_365 | mean     | std      | units | TO      |
|-----|---------------------|----------------|-----------|---------|----------|----------|-----|-----|-----|-----|-----|-------|-------|-------|-------|-------|-------|----------|----------|-------|---------|
| 0   | CA_1-HOBBIES_1\_002 | HOBBIES_1\_002 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 0     | 0     | 1     | 1     | 0     | 0     | 0.175342 | 0.459270 | 64    | 254.08  |
| 1   | CA_1-HOBBIES_1\_004 | HOBBIES_1\_004 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 1     | 3     | 0     | 1     | 2     | 3     | 1.282192 | 1.504512 | 468   | 2031.12 |
| 2   | CA_1-HOBBIES_1\_005 | HOBBIES_1\_005 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 0     | 0     | 0     | 0     | 0     | 0     | 0.794521 | 1.543854 | 290   | 800.70  |
| 3   | CA_1-HOBBIES_1\_008 | HOBBIES_1\_008 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 12  | 15  | 0   | 0   | ... | 0     | 9     | 14    | 0     | 8     | 18    | 6.854795 | 8.556423 | 2502  | 1237.92 |
| 4   | CA_1-HOBBIES_1\_009 | HOBBIES_1\_009 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 2   | 0   | 7   | 3   | ... | 0     | 1     | 0     | 0     | 3     | 0     | 1.816438 | 2.486613 | 663   | 1151.04 |

5 rows × 375 columns

### Coefficient of Variation<a href="#Coefficient-of-Variation" class="anchor-link">¶</a>

\| CV = sigma/mu

In \[10\]:

    # Coefficient of Variation
    df['CV'] = df['std']/df['mean']
    df.head()

Out\[10\]:

|     | id                  | item_id        | dept_id   | cat_id  | store_id | state_id | d_1 | d_2 | d_3 | d_4 | ... | d_361 | d_362 | d_363 | d_364 | d_365 | mean     | std      | units | TO      | CV       |
|-----|---------------------|----------------|-----------|---------|----------|----------|-----|-----|-----|-----|-----|-------|-------|-------|-------|-------|----------|----------|-------|---------|----------|
| 0   | CA_1-HOBBIES_1\_002 | HOBBIES_1\_002 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 0     | 1     | 1     | 0     | 0     | 0.175342 | 0.459270 | 64    | 254.08  | 2.619273 |
| 1   | CA_1-HOBBIES_1\_004 | HOBBIES_1\_004 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 3     | 0     | 1     | 2     | 3     | 1.282192 | 1.504512 | 468   | 2031.12 | 1.173391 |
| 2   | CA_1-HOBBIES_1\_005 | HOBBIES_1\_005 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 0   | 0   | 0   | 0   | ... | 0     | 0     | 0     | 0     | 0     | 0.794521 | 1.543854 | 290   | 800.70  | 1.943127 |
| 3   | CA_1-HOBBIES_1\_008 | HOBBIES_1\_008 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 12  | 15  | 0   | 0   | ... | 9     | 14    | 0     | 8     | 18    | 6.854795 | 8.556423 | 2502  | 1237.92 | 1.248239 |
| 4   | CA_1-HOBBIES_1\_009 | HOBBIES_1\_009 | HOBBIES_1 | HOBBIES | CA_1     | CA       | 2   | 0   | 7   | 3   | ... | 1     | 0     | 0     | 3     | 0     | 1.816438 | 2.486613 | 663   | 1151.04 | 1.368950 |

5 rows × 376 columns

------------------------------------------------------------------------

### Segmentation<a href="#Segmentation" class="anchor-link">¶</a>

#### ABC Analysis + Demand Variability<a href="#ABC-Analysis-+-Demand-Variability" class="anchor-link">¶</a>

\| @ SKU Level

In \[11\]:

    # ABC SKU-LEVEL
    df_abc = df[df['cat_id']=='HOBBIES'].drop(['mean', 'std', 'CV'], axis = 1).copy()
    df_abc = df_abc.groupby(['item_id', 'dept_id', 'cat_id']).sum()

    # Calculate mean and standard deviation
    # Mean
    df_abc['mean'] = df_abc[COLS_DATE].mean(axis = 1)
    # Filter out the reference withou sales
    df_abc = df_abc[df_abc['mean']>0]
    # Standard
    df_abc['std'] = df_abc[COLS_DATE].std(axis = 1)
    # Coefficient of Variation
    df_abc['CV'] = df_abc['std']/df_abc['mean']
    df_abc.reset_index(inplace = True)
    # Normalility Test
    df_abc['Normality_p'] = df_abc[COLS_DATE].apply(lambda row : stats.shapiro(row)[1], axis = 1)
    alpha = 0.001
    df_abc['Not_Normal'] = df_abc['Normality_p'] < alpha

    # # ABC SKU-LEVEL
    df_abc = df_abc.drop(COLS_DATE, axis =1).copy()
    df_abc['TO%'] = (100*df_abc['TO']/df_abc['TO'].sum())
    # Sort 
    df_abc.sort_values(['TO%'], ascending = False, inplace = True, ignore_index=True)
    df_abc['TO%_CS'] = df_abc['TO%'].cumsum()

    # A, B, C on SKU Number
    n_sku = len(df_abc)
    n_a, n_b = int(0.05*n_sku), int(0.5*n_sku)
    df_abc['SKU_ID'] = pd.Series(range(1, len(df_abc))).astype(int)
    df_abc['SKU_%'] = (100 * pd.Series(range(1, len(df_abc))) / len(df_abc))
    df_abc['ABC'] = pd.Series(range(len(df_abc))).apply(lambda t: 'A' if t <= n_a-1 else 'B' if t <= n_b-1 else 'C')
    # A, B, C on turnover
    to_a, to_b = df_abc[df_abc['SKU_ID']==n_a]['TO%'].max(), df_abc[df_abc['SKU_ID']==n_b]['TO%'].max()

    print("{:,} unique SKU".format(len(df_abc)))
    df_abc.head()

    317 unique SKU

Out\[11\]:

|     | item_id        | dept_id   | cat_id  | units | TO       | mean      | std       | CV       | Normality_p  | Not_Normal | TO%      | TO%\_CS  | SKU_ID | SKU\_%   | ABC |
|-----|----------------|-----------|---------|-------|----------|-----------|-----------|----------|--------------|------------|----------|----------|--------|----------|-----|
| 0   | HOBBIES_1\_099 | HOBBIES_1 | HOBBIES | 4139  | 61800.95 | 11.339726 | 7.818314  | 0.689462 | 3.602743e-12 | True       | 2.157121 | 2.157121 | 1.0    | 0.315457 | A   |
| 1   | HOBBIES_1\_236 | HOBBIES_1 | HOBBIES | 4855  | 57876.15 | 13.301370 | 15.026977 | 1.129732 | 1.162563e-19 | True       | 2.020128 | 4.177249 | 2.0    | 0.630915 | A   |
| 2   | HOBBIES_1\_141 | HOBBIES_1 | HOBBIES | 3693  | 55265.78 | 10.117808 | 7.187431  | 0.710374 | 1.462927e-12 | True       | 1.929015 | 6.106264 | 3.0    | 0.946372 | A   |
| 3   | HOBBIES_1\_345 | HOBBIES_1 | HOBBIES | 6690  | 50257.55 | 18.328767 | 6.358465  | 0.346912 | 1.292406e-01 | False      | 1.754206 | 7.860470 | 4.0    | 1.261830 | A   |
| 4   | HOBBIES_1\_226 | HOBBIES_1 | HOBBIES | 3332  | 49797.61 | 9.128767  | 3.615803  | 0.396089 | 4.752485e-05 | True       | 1.738152 | 9.598622 | 5.0    | 1.577287 | A   |

\| Distribution by Value

In \[12\]:

    # Distribution by Value
    ax = plt.gca()
    df_abc.plot(figsize=(12, 8), x='SKU_%', y='TO%_CS', ax =ax, grid = True)
    # ABC
    # 20%, 50% of SKU Number
    ax.axvline(5 , color="red", linestyle="-", linewidth = 1.0)
    ax.axvline(20 , color="red", linestyle="-", linewidth = 1.0)
    # 20%, 50% of SKU Number
    ax.axhline(80 , color="blue", linestyle="--", linewidth = 1.0)
    ax.axhline(95 , color="blue", linestyle="--", linewidth = 1.0)
    plt.xlabel('Percentage of SKU (%)')
    plt.xticks(rotation=90)
    plt.ylabel('Percentage of the Annual Turnover (%)')
    plt.title('ABC Analysis: Distribution by Turnover (Sales Value in $)')
    plt.show()

![](...%0A)

### Segmentation by Demand Variability<a href="#Segmentation-by-Demand-Variability" class="anchor-link">¶</a>

\| Coefficient of Variation = f(% TO)

In \[16\]:

    # Bar Chart
    ax = plt.gca()
    colors = {'A':'red', 'B':'green', 'C':'blue'}
    # Remove Outliers
    df_plot = df_abc[df_abc['CV']<4].copy()
    df_plot.plot.scatter(figsize=(12, 8), x='TO%', y='CV', color=df_plot['ABC'].map(colors), ax =ax, grid = True)
    # ABC
    # A, B and C
    ax.axvline(to_a , color="red", linestyle="-", linewidth = 1.0)
    ax.axvline(to_b , color="red", linestyle="-", linewidth = 1.0)
    # 20%, 50% of SKU Number
    ax.axhline(1 , color="blue", linestyle="--", linewidth = 1.0)
    plt.xlabel('Percentage of Turnover (%)')
    plt.xticks(rotation=90)
    plt.ylabel('Coefficient of Variation')
    plt.title('Distribution by Demand Variability')
    plt.show()

![](...%0A)

### Distribution for Important Products with high variability<a href="#Distribution-for-Important-Products-with-high-variability" class="anchor-link">¶</a>

\| Example of product with High COV

In \[18\]:

    # ABC SKU-LEVEL
    df_dist = df[df['cat_id']=='HOBBIES'].drop(['mean', 'std', 'CV'], axis = 1).copy()
    df_dist = pd.DataFrame(df_dist.groupby(['item_id', 'dept_id', 'cat_id']).sum())
    df_dist.reset_index(inplace = True)

    # Item A
    item_high = 'HOBBIES_1_235'
    df_dist = df_dist[df_dist['item_id']==item_high][COLS_DATE].T
    df_dist.columns = ['Units']

    # Simple histogram
    df_dist['Units'].hist(figsize=(10,7), edgecolor='black', grid = False, bins = df_dist['Units'].nunique()) # , linewidth=1.2 , bins = 30
    plt.xlabel('Sales Volume (Units/Day)')
    plt.ylabel('Number of Days per Year')
    plt.title('Distribution of Sales for item: {}'.format(item_high))
    plt.show()

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAmQAAAG5CAYAAAAgWSjQAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90%0AbGliIHZlcnNpb24zLjMuNCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8QVMy6AAAACXBIWXMAAAsT%0AAAALEwEAmpwYAAAtNElEQVR4nO3deZhkZX328e8toKhsIiMOyzCKaIIYAUdFRcU1oIhLFCWKgORF%0AjRtKEhQxGo2Ju3F51RfFgBuC4oKoETWCSyKyiOybCMIwDAPIKiIMv/ePcxqKprq7Zuiq0zP1/VxX%0AXX3qOXXO+dWpmum7n+csqSokSZLUnXt1XYAkSdK4M5BJkiR1zEAmSZLUMQOZJElSxwxkkiRJHTOQ%0ASZIkdcxAJkmS1DEDmdRHks8keccsrWtBkhuTrNE+Pz7J383Gutv1fT/JXrO1vhXY7r8muSrJFbO8%0A3p2SXDab62zX+9okS9vP4oGzsL6zkux0zyuTJAOZxlCSi5PcnOSGJNcm+Z8kr0lyx7+HqnpNVb1n%0AwHU9c7rXVNXvq2qdqlo+C7W/K8mXJq1/l6o6/J6uewXrWAAcAGxdVQ+e4jUHJfldG4AuS3LkKGuc%0AVMtawEeAZ7efxdX3dJ1V9ciqOr5d/90+l1HqF/L7Bdskuyb5VZKbklyd5MtJNuuZv3eS5e1ndmOS%0Ai5K8tmf+wiTVM39pkk+1+3fiNXf8m+izvonHJu38Hdt/f9cluSbJL5I8dob3Oj/JMUkub2tZOOA+%0A+lCSC9p/9+cmeWXPvI3abV/d/p/wv0meNM1+udEwrtlmINO4el5VrQtsAbwPOBA4dLY3kmTN2V7n%0AHLEAuLqqruw3s+2x2xN4ZlWtAywCfjzC+ibbGFgbOGtFF0xjlf+/MsmLga8A/wFsBDwSuAX4eZIH%0A9Lz0f9vQug7wN8AHkmw3aXUbtPMfBTwBeN00m75jfT2Py5OsBxwLfALYENgU+Je2puncDvxXW9uK%0AuAl4HrA+sBfwsSRPbOfdCLwKmAc8AHg/8J1J/34nv4/jV3D70rRW+f9kpHuiqq6rqmOAlwJ7JdkG%0AIMlhSf61nd4oybHtX87XJPlZknsl+SJNMPlO+xfzP/X0IOyb5PfAf/e09f7nvmXbU3F9km8n2bDd%0AVr9ejYuTPDPJzsBBwEvb7f2mnX9H70hb18FJLklyZZIvJFm/nTdRx15Jfp9muPHtU+2bJOu3yy9r%0A13dwu/5nAj8ENmnrOKzP4o8FflBVv2338xVVdUjPuvdJck7bW3FRkldPU8cmSY5u6/hdkjf2zHtc%0AkpPb/bg0yUf6LP9w4Lz26bVJ/rttf2KSk9remZN6fjlP7NP3JvkF8EfgoX3WO9Pnsn6SQ5MsSbI4%0AzRDvxLD13m2PzEfb79VFbT17J7m0/exmbRg6SYAPA/9aVV+pqpur6grg72jCyJv7LVdVvwbOAf5y%0AivlX0nwXtl6Jsh7eruOIqlre1nRcVZ0+3UJVtbSqPgWctCIbq6p3VtW5VXV7VZ0I/IwmTFJVf6qq%0A86rqdiDAcppgtuFKvC9ppRjIJKCqfgVcBjy5z+wD2nnzaHpaDmoWqT2B39P0tq1TVR/oWeapNL/E%0A/nqKTb6S5i/y+cBtwMcHqPG/gH8Djmy39+g+L9u7fTyNJkSsA3xy0mt2BB4BPAP45yR9f9nS9Fys%0A367nqW3N+1TVj4BdgMvbOvbus+wvgVcm+cckiyaCSI8rgV2B9YB9gI8m2X7yStqeqe8Av6HpQXkG%0AsH+Sif36MeBjVbUesCVw1OR1VNX5NL1B0PTsPL0NwN+l2e8PpBnO/G7uemzZnsB+wLrAJVPso+k+%0Al8NoPtuHAdsBz6YJQBMeD5zebv8rwFdpguzDgFcAn0yyTrsf/jbJtEFlBo+g+ePha5Nqvx04GnhW%0Av4XSDB8+HDh5ivmb0HzHf7kSNZ0PLE9yeJJdctdeuqFKcl+afX3WpPbTgT8BxwCfm9QDvF37R8z5%0ASd6R1bf3Wx0xkEl3upz+fxHfShOctqiqW6vqZ1VVM6zrXVV1U1XdPMX8L1bVmVV1E/AOYPc+oWVl%0AvBz4SFVdVFU3Am8DXjbpl8e/tL0Rv6EJOncLdm0tLwPeVlU3VNXFND0sew5SRFV9CXgDzS/rE4Ar%0AkxzYM/+7VfXbapwAHEf/MPxYYF5Vvbuq/lxVFwGfbWuD5rN5WJKNqurGqho0GDwXuKCqvlhVt1XV%0AEcC5NENaEw6rqrPa+bcOuF4AkmwMPAfYv/0eXAl8tKdugN9V1X+2xxYeCWwOvLuqbqmq44A/04Qz%0A2l6tv5phsx9ve9uuTXItzXDghI3an0v6LLekZz7ADu06bgB+BXwRuGDSMle121hMMxT49Wnq2qG3%0AriQTvabX0/xxUDSf6bI0x4ZtPMP7nA2fofnu/6C3sd3H6wF/C/y8Z9ZPgW2AB9EMle4B/OMI6tQY%0AMZBJd9oUuKZP+weBC4Hj2qGltw6wrktXYP4lwFrc9ZfiytqEu/bmXAKsSdOzN6H3rMg/0vSiTbZR%0AW9PkdW06aCFV9eWqeiawAfAa4D0TPVttj8gv0wwBX0sTXvq9/y1ohkZ7g8ZBPe9nX5oenHPbYcdd%0AByxv8n7q9/5m+gynswXN/lvSU/f/o/mFPmFpz/TN0AzHTWrr99lM5Y1VtcHEg6YHcsJV7c/5fZab%0A3zMf4JftOtYFHkzTu/hvk5bZqN3G/YBfMCnYTPLL3rqqasuJGVV1TlXtXVWb0QSeTWiOcRuaJB9s%0At7V7vz+s2uHLI4C3Jnl023ZRVf2uHe48A3g38OJh1qnxYyCTuGNoZlPu+lcxAG0P0QFV9VBgN+At%0ASZ4xMXuKVc7Ug7Z5z/QCmp6eq2h6G+7XU9caNEOlg673cpow0Lvu27jrL/9BXNXWNHldi1dwPbS9%0Ail+jGZ7bJsl9aIbJPgRs3P5i/x7NsTuTXUrTk9T7C33dqnpOu+4LqmoPmqDzfuDrSe4/QFmT91O/%0A9zfTvu41+bWX0hycvlFP3etV1SP7LDsK59EMu7+kt7EdEv4bpjjhog2IR3PXnsPe+TfTDM3ukOQe%0A/UFRVee269rmnqxnOkn+hWa4/dltD9101qLPsYOtov/3VVppBjKNtSTrtb0qXwW+1P71O/k1uyZ5%0AWHtg9HU0B/ze3s5eytT/aU/nFUm2TnI/mr+2v94OXZ0PrJ3kuWkuJXAwcJ+e5ZYCCzP1WX9HAG9O%0A8pD2+KOJY5tuW5Hi2lqOAt6bZN0kWwBvAQa6tEN7cPpz22XvlWQXmp6WE4F7t+9pGXBbO+/ZU6zq%0AV8ANSQ5Mct8kayTZpg3QJHlFknntsVDXtsvcPsW6en0PeHh7bNaaSV5Kc2D6sTMsN5W7fC5VtYRm%0AGPbD7XfsXkm2TPLUlVz/PdL2BP0DcHD7ntdO8mDgczRDdB/tt1x7TN0LmeLs1DZc70nT67pClxJJ%0A8hdJDkh72Y0km9MMBc447Jxkbe78d3Gf9vlMy7yNZijymTXpsidJdkhzCY57t9+zA2l6YU9s5+8y%0AMZSa5C9oDjP49qDvVRqEgUzj6jvtMTKXAm+nOah7nyleuxXwI5qz0f4X+FRV/aSd9+80v+SuTfIP%0AK7D9L9L0BlxBczmGN0Jz1ifw9zS/KCeOz+k963LioOyrk5zaZ72fb9f9U+B3NAcov2EF6ur1hnb7%0AF9H0HH6lXf8grqcZWvw9TVD6APDaqvp5Vd1A836PAv5A80vymH4raYPhrsC2NO/nKpp9s377kp2B%0As5LcSHOA/8umOW6vd71Xt+s9gCZI/BOwa1VdNe2CU+v3ubySJnyeTfM+v07/IcMZJXl5khW+ZEev%0AqjqSJjy9meY9nw3cF3jSpIDyhLTX2qI5w3IZd/8OXdvOX0pzpuJu0xxXecf6eh6PBW6gObHhxCQ3%0A0QSxM2k+k5ncTPPvEZpj/2b8zGn+OFkAXNhTx0HtvPsA/5dmvyymGUJ/blVd3s5/BnB6W+f3gG9w%0A92Fc6R7JzMcmS5IkaZjsIZMkSeqYgUySNKekuZfs5GHOG5N8Zobl+i1zY5J+l1SR5hSHLCVJkjq2%0ASl9peKONNqqFCxd2XYYkSdKMTjnllKuqal6/eat0IFu4cCEnn9z3jh6SJElzSpIpb8PmMWSSJEkd%0AM5BJkiR1zEAmSZLUMQOZJElSxwxkkiRJHTOQSZIkdcxAJkmS1DEDmSRJUscMZJIkSR0zkEmSJHXM%0AQCZJktQxA5kkSVLHDGSSJEkdM5BJkiR1zEAmSZLUMQOZJElSxwxkA5i/2QKSzOpj/mYLun5bkiRp%0Ajliz6wJWBVcsvpQtDjx2Vtd5yft3ndX1SZKkVZc9ZJIkSR0bWiBLsnmSnyQ5O8lZSd7Utm+Y5IdJ%0ALmh/PqBtT5KPJ7kwyelJth9WbZIkSXPJMHvIbgMOqKqtgR2A1yXZGngr8OOq2gr4cfscYBdgq/ax%0AH/DpIdYmSZI0ZwwtkFXVkqo6tZ2+ATgH2BR4PnB4+7LDgRe0088HvlCNXwIbJJk/rPokSZLmipEc%0AQ5ZkIbAdcCKwcVUtaWddAWzcTm8KXNqz2GVt2+R17Zfk5CQnL1u2bHhFS5IkjcjQA1mSdYCjgf2r%0A6vreeVVVQK3I+qrqkKpaVFWL5s2bN4uVSpIkdWOogSzJWjRh7MtV9Y22eenEUGT788q2fTGwec/i%0Am7VtkiRJq7VhnmUZ4FDgnKr6SM+sY4C92um9gG/3tL+yPdtyB+C6nqFNSZKk1dYwLwz7JGBP4Iwk%0Ap7VtBwHvA45Ksi9wCbB7O+97wHOAC4E/AvsMsTZJkqQ5Y2iBrKp+DmSK2c/o8/oCXjeseiRJkuYq%0Ar9QvSZLUMQOZJElSxwxkkiRJHTOQSZIkdcxAJkmS1DEDmSRJUscMZJIkSR0zkEmSJHXMQCZJktQx%0AA5kkSVLHDGSSJEkdM5BJkiR1zEAmSZLUMQOZJElSxwxkkiRJHTOQSZIkdcxAJkmS1DEDmSRJUscM%0AZJIkSR0zkEmSJHXMQCZJktQxA5kkSVLHDGSSJEkdM5BJkiR1zEAmSZLUMQOZJElSxwxkkiRJHTOQ%0ASZIkdcxAJkmS1DEDmSRJUscMZJIkSR0zkEmSJHXMQCZJktQxA5kkSVLHDGSSJEkdG1ogS/L5JFcm%0AObOn7cgkp7WPi5Oc1rYvTHJzz7zPDKsuSZKkuWbNIa77MOCTwBcmGqrqpRPTST4MXNfz+t9W1bZD%0ArEeSJGlOGlogq6qfJlnYb16SALsDTx/W9iVJklYVXR1D9mRgaVVd0NP2kCS/TnJCkidPtWCS/ZKc%0AnOTkZcuWDb9SSZKkIesqkO0BHNHzfAmwoKq2A94CfCXJev0WrKpDqmpRVS2aN2/eCEqVJEkarpEH%0AsiRrAi8Cjpxoq6pbqurqdvoU4LfAw0ddmyRJUhe66CF7JnBuVV020ZBkXpI12umHAlsBF3VQmyRJ%0A0sgN87IXRwD/CzwiyWVJ9m1nvYy7DlcCPAU4vb0MxteB11TVNcOqTZIkaS4Z5lmWe0zRvneftqOB%0Ao4dViyRJ0lzmlfolSZI6ZiCTJEnqmIFMkiSpYwYySZKkjhnIJEmSOmYgkyRJ6piBTJIkqWMGMkmS%0ApI4ZyCRJkjpmIJMkSeqYgUySJKljBjJJkqSOGcgkSZI6ZiCTJEnqmIFMkiSpYwYySZKkjhnIJEmS%0AOmYgkyRJ6piBTJIkqWMGMkmSpI4ZyCRJkjpmIJMkSeqYgUySJKljBjJJkqSOGcgkSZI6ZiCTJEnq%0AmIFMkiSpYwYySZKkjhnIJEmSOmYgkyRJ6piBTJIkqWMGMkmSpI4ZyCRJkjpmIJMkSeqYgUySJKlj%0ABjJJkqSODS2QJfl8kiuTnNnT9q4ki5Oc1j6e0zPvbUkuTHJekr8eVl2SJElzzTB7yA4Ddu7T/tGq%0A2rZ9fA8gydbAy4BHtst8KskaQ6xNkiRpzhhaIKuqnwLXDPjy5wNfrapbqup3wIXA44ZVmyRJ0lzS%0AxTFkr09yejuk+YC2bVPg0p7XXNa23U2S/ZKcnOTkZcuWDbtWSZKkoRt1IPs0sCWwLbAE+PCKrqCq%0ADqmqRVW1aN68ebNcniRJ0uiNNJBV1dKqWl5VtwOf5c5hycXA5j0v3axtkyRJWu2NNJAlmd/z9IXA%0AxBmYxwAvS3KfJA8BtgJ+NcraJEmSurLmsFac5AhgJ2CjJJcB7wR2SrItUMDFwKsBquqsJEcBZwO3%0AAa+rquXDqk2SJGkuGVogq6o9+jQfOs3r3wu8d1j1SJIkzVVeqV+SJKljBjJJkqSOGcgkSZI6ZiCT%0AJEnqmIFMkiSpYwYySZKkjhnIJEmSOmYgkyRJ6piBTJIkqWMGMkmSpI4ZyCRJkjo2bSBLskaSD42q%0AGEmSpHE0bSCrquXAjiOqRZIkaSytOcBrfp3kGOBrwE0TjVX1jaFVJUmSNEYGCWRrA1cDT+9pK8BA%0AJkmSNAtmDGRVtc8oCpEkSRpXMwayJGsD+wKPpOktA6CqXjXEuiRJksbGIJe9+CLwYOCvgROAzYAb%0AhlmUJEnSOBkkkD2sqt4B3FRVhwPPBR4/3LIkSZLGxyCB7Nb257VJtgHWBx40vJIkSZLGyyBnWR6S%0A5AHAO4BjgHWAfx5qVZIkSWNkkLMsP9dOngA8dLjlSJIkjZ8ZhyyTbJzk0CTfb59vnWTf4ZcmSZI0%0AHgY5huww4AfAJu3z84H9h1SPJEnS2BkkkG1UVUcBtwNU1W3A8qFWJUmSNEYGCWQ3JXkgze2SSLID%0AcN1Qq5IkSRojUx7Un+SlVXUk8Baasyu3TPILYB7w4hHVJ0mStNqb7izLPZPsA/w98FTgEUCA86rq%0A1mmWkyRJ0gqYcsiyqnYFPgN8F3gbsBS4HFg3yYajKU+SJGn1N+11yKrqW0l+B/yU5gbjNTELr0km%0ASZI0K6Y7huw+wME0x4u9vKqOHVlVkiRJY2S6syxPB9YAtjeMSZIkDc90Q5YvrKqzR1aJJEnSmJru%0AoH7DmCRJ0ggMcmFYSZIkDdG0gSzJGknePKpiJEmSxtG0gayqlgN7rMyKk3w+yZVJzuxp+2CSc5Oc%0AnuSbSTZo2xcmuTnJae3jMyuzTUmSpFXRIEOWv0jyySRPTrL9xGOA5Q4Ddp7U9kNgm6r6K+B8mgvO%0ATvhtVW3bPl4zUPWSJEmrgWkvDNvatv357p62Ap4+3UJV9dMkCye1Hdfz9Jd4T0xJkqSZA1lVPW1I%0A234VcGTP84ck+TVwPXBwVf2s30JJ9gP2A1iwYMGQSpMkSRqdGYcsk2yc5NAk32+fb51k33uy0SRv%0AB24Dvtw2LQEWVNV2wFuAryRZr9+yVXVIVS2qqkXz5s27J2VIkiTNCYMcQ3YY8ANgk/b5+cD+K7vB%0AJHsDu9LcjqkAquqWqrq6nT4F+C3w8JXdhiRJ0qpkkEC2UVUdBdwOUFW3ActXZmNJdgb+Cditqv7Y%0A0z4vyRrt9EOBrYCLVmYbkiRJq5pBDuq/KckDaQ7kJ8kOwHUzLZTkCGAnYKMklwHvpDmr8j7AD5MA%0A/LI9o/IpwLuT3EoT/F5TVdes+NuRJEla9QwSyN4CHANsmeQXwDwGODuyqvpdv+zQKV57NHD0ALVI%0AkiStdgY5y/LUJE8FHgEEOK+qbh16ZZIkSWNixkCWZG3g74EdaYYtf5bkM1X1p2EXJ0mSNA4GGbL8%0AAnAD8In2+d8CXwReMqyiJEmSxskggWybqtq65/lPkpw9rIIkSZLGzSCXvTi1PbMSgCSPB04eXkmS%0AJEnjZZAesscA/5Pk9+3zBcB5Sc4Aqr1RuCRJklbSIIFs56FXIUmSNMYGuezFJaMoRJIkaVwNcgyZ%0AJEmShshAJkmS1LEZA1mS+ye5Vzv98CS7JVlr+KVJkiSNh0F6yH4KrJ1kU+A4YE/gsGEWJUmSNE4G%0ACWSpqj8CLwI+VVUvAR453LIkSZLGx0CBLMkTgJcD323b1hheSZIkSeNlkED2JuBtwDer6qwkDwV+%0AMtyyJEmSxscgF4b9Q1XtNvGkqi4C3ji8kiRJksbLID1kn0ryqyR/n2T9oVckSZI0ZmYMZFX1ZOAV%0AwObAKUm+kuTZQ69MkiRpTAx0YdiqOh84GDgQeCrwsSTnJnnRMIuTJEkaB4NcGPavknwUOAd4OvC8%0AqvrLdvqjQ65PkiRptTfIQf2fAD4HHFRVN080VtXlSQ4eWmWSJEljYsZAVlVPnWbeF2e3HEmSpPEz%0AYyBLshXw78DWwNoT7VX10CHWJUmSNDYGOaj/P4FPA7cBTwO+AHxpmEVJkiSNk0EC2X2r6sc097S8%0ApKreBTx3uGVJkiSNj0EO6r8lyb2AC5K8HlgMrDPcsiRJksbHoPeyvB/N7ZIeA+wJ7DXMoiRJksbJ%0AIGdZntRO3gjsM9xyJEmSxs+0PWRJ9kpyapKb2sfJSV45quIkSZLGwZQ9ZEn2AvYH3gKcCgTYHvhg%0AkvIaZJIkSbNjuh6y1wIvrKqfVNV1VXVtVf038DfA60ZTniRJ0upvukC2XlVdPLmxbVtvWAVJkiSN%0Am+kC2c0rOU+SJEkrYLqzLP8yyel92gN42yRJkqRZMm0gG1kVkiRJY2zKQFZVl9zTlSf5PLArcGVV%0AbdO2bQgcCSwELgZ2r6o/JAnwMeA5wB+Bvavq1HtagyRJ0lw3yJX674nDgJ0ntb0V+HFVbQX8uH0O%0AsAuwVfvYj+aG5pIkSau9oQayqvopcM2k5ucDh7fThwMv6Gn/QjV+CWyQZP4w65MkSZoLpgxkSX7c%0A/nz/LG9z46pa0k5fAWzcTm8KXNrzusvatsl17dfeMeDkZcuWzXJpkiRJozfdQf3zkzwR2C3JV2nO%0ArrzDbBzfVVWVpFZwmUOAQwAWLVq0QstKkiTNRdMFsn8G3gFsBnxk0rwCnr6S21yaZH5VLWmHJK9s%0A2xcDm/e8brO2TZIkabU25ZBlVX29qnYBPlBVT5v0WNkwBnAMsFc7vRfw7Z72V6axA3Bdz9CmJEnS%0Aamu6HjIAquo9SXYDntI2HV9Vxw6y8iRHADsBGyW5DHgn8D7gqCT7ApcAu7cv/x7NJS8upLnsxT4r%0A8D4kSZJWWTMGsiT/DjwO+HLb9KYkT6yqg2Zatqr2mGLWM/q8tvCm5ZIkaQzNGMiA5wLbVtXtAEkO%0AB34NzBjIJEmSNLNBr0O2Qc/0+kOoQ5IkaWwN0kP278Cvk/yE5tIXT+HOq+tLkiTpHhrkoP4jkhwP%0APLZtOrCqrhhqVZIkSWNkkB4y2stPHDPkWiRJksbSsG8uLkmSpBkYyCRJkjo2bSBLskaSc0dVjCRJ%0A0jiaNpBV1XLgvCQLRlSPJEnS2BnkoP4HAGcl+RVw00RjVe02tKokSZLGyCCB7B1Dr0KSJGmMDXId%0AshOSbAFsVVU/SnI/YI3hlyZJkjQeZjzLMsn/Ab4O/L+2aVPgW0OsSZIkaawMctmL1wFPAq4HqKoL%0AgAcNsyhJkqRxMkggu6Wq/jzxJMmaQA2vJEmSpPEySCA7IclBwH2TPAv4GvCd4ZYlSZI0PgYJZG8F%0AlgFnAK8GvgccPMyiJEmSxskgZ1nenuRw4ESaocrzqsohS0mSpFkyYyBL8lzgM8BvgQAPSfLqqvr+%0AsIuTJEkaB4NcGPbDwNOq6kKAJFsC3wUMZJIkSbNgkGPIbpgIY62LgBuGVI8kSdLYmbKHLMmL2smT%0Ak3wPOIrmGLKXACeNoDZJkqSxMN2Q5fN6ppcCT22nlwH3HVpFkiRJY2bKQFZV+4yyEEmSpHE1yFmW%0ADwHeACzsfX1V7Ta8siRJksbHIGdZfgs4lObq/LcPtRpJkqQxNEgg+1NVfXzolUiSJI2pQQLZx5K8%0AEzgOuGWisapOHVpVkiRJY2SQQPYoYE/g6dw5ZFntc0mSJN1DgwSylwAPrao/D7sYSZKkcTTIlfrP%0ABDYYch2SJElja5Aesg2Ac5OcxF2PIfOyF5IkSbNgkED2zqFXIUmSNMZmDGRVdcIoCpEkSRpXg1yp%0A/waasyoB7g2sBdxUVesNszBJkqRxMUgP2boT00kCPB/YYZhFSZIkjZNBzrK8QzW+Bfz1ym4wySOS%0AnNbzuD7J/knelWRxT/tzVnYbkiRJq5JBhixf1PP0XsAi4E8ru8GqOg/Ytl33GsBi4JvAPsBHq+pD%0AK7tuSZKkVdEgZ1k+r2f6NuBimmHL2fAM4LdVdUkzGipJkjR+BjmGbJ8hbv9lwBE9z1+f5JXAycAB%0AVfWHyQsk2Q/YD2DBggVDLE2SJGk0pgxkSf55muWqqt5zTzac5N7AbsDb2qZPA++hOaPzPcCHgVf1%0A2fAhwCEAixYtqsnzJUmSVjXTHdR/U58HwL7AgbOw7V2AU6tqKUBVLa2q5VV1O/BZ4HGzsA1JkqQ5%0Ab8oesqr68MR0knWBN9EceP9Vmt6re2oPeoYrk8yvqiXt0xfS3ENTkiRptTftMWRJNgTeArwcOBzY%0Avt9xXSsqyf2BZwGv7mn+QJJtaYYsL540T5IkabU13TFkHwReRHO81qOq6sbZ2mhV3QQ8cFLbnrO1%0AfkmSpFXJdMeQHQBsAhwMXN5ewPX6JDckuX405UmSJK3+pjuGbIWu4i9JkqSVY+iSJEnqmIFMkiSp%0AYwYySZKkjhnIJEmSOmYgkyRJ6piBTJIkqWMGMkmSpI4ZyCRJkjpmIJMkSeqYgUySJKljBjJJkqSO%0AGcgkSZI6ZiCTJEnqmIFMkiSpYwYySZKkjhnIJEmSOmYgkyRJ6piBTJIkqWMGMkmSpI4ZyCRJkjpm%0AIJMkSeqYgUySJKljBjJJkqSOGcgkSZI6ZiCTJEnqmIFMkiSpYwYySZKkjhnIJEmSOmYgkyRJ6piB%0ATJIkqWMGMkmSpI4ZyCRJkjpmIJMkSerYml1tOMnFwA3AcuC2qlqUZEPgSGAhcDGwe1X9oasaJUmS%0ARqHrHrKnVdW2VbWoff5W4MdVtRXw4/a5JEnSaq3rQDbZ84HD2+nDgRd0V4okSdJodBnICjguySlJ%0A9mvbNq6qJe30FcDG3ZQmSZI0Op0dQwbsWFWLkzwI+GGSc3tnVlUlqckLteFtP4AFCxaMplJJkqQh%0A6qyHrKoWtz+vBL4JPA5YmmQ+QPvzyj7LHVJVi6pq0bx580ZZsiRJ0lB0EsiS3D/JuhPTwLOBM4Fj%0AgL3al+0FfLuL+iRJkkapqyHLjYFvJpmo4StV9V9JTgKOSrIvcAmwe0f1SZIkjUwngayqLgIe3af9%0AauAZo69IkiSpO3PtsheSJEljx0AmSZLUMQOZJElSxwxkkiRJHTOQSZIkdcxAJkmS1DEDmSRJUscM%0AZJIkSR0zkEmSJHXMQCZJktQxA5kkSVLHDGSSJEkdM5BJkiR1zEAmSZLUMQOZJElSxwxkkiRJHTOQ%0ASZIkdcxAJkmS1DEDmSRJUscMZJIkSR0zkEmSJHXMQCZJktQxA5kkSVLHDGSSJEkdM5BJkiR1zEAm%0ASZLUMQOZJElSxwxk0mpg/mYLSDKrj/mbLej6bUnS2Fiz6wIk3XNXLL6ULQ48dlbXecn7d53V9UmS%0ApmYPmSRJUscMZJIkSR0zkEmSJHXMQCZJktQxA5kkSVLHDGSSJEkdM5BJkiR1bOSBLMnmSX6S5Owk%0AZyV5U9v+riSLk5zWPp4z6tokSZK60MWFYW8DDqiqU5OsC5yS5IftvI9W1Yc6qEmSJKkzIw9kVbUE%0AWNJO35DkHGDTUdchSZI0V3R6DFmShcB2wIlt0+uTnJ7k80keMMUy+yU5OcnJy5YtG1WpkiRJQ9NZ%0AIEuyDnA0sH9VXQ98GtgS2JamB+3D/ZarqkOqalFVLZo3b96oypUkSRqaTgJZkrVowtiXq+obAFW1%0AtKqWV9XtwGeBx3VRmyRJ0qh1cZZlgEOBc6rqIz3t83te9kLgzFHXJkmS1IUuzrJ8ErAncEaS09q2%0Ag4A9kmwLFHAx8OoOapMkSRq5Ls6y/DmQPrO+N+paJEmS5gKv1C9JktQxA5kkSVLHDGSSJEkdM5BJ%0AkiR1zEAmSZLUMQOZJElSxwxkkiRJHTOQSZIkdcxAJkmS1DEDmSRJUscMZJIkSR0zkEmSJHXMQCZJ%0AktQxA5kkSVLHDGSSJEkdM5BJkiR1zEAmSZPM32wBSWb1MX+zBV2/LUlz2JpdFyBpfMzfbAFXLL50%0AVtf54E03Z8llv5/VdV6x+FK2OPDYWV3nJe/fdVbXJ2n1YiCTNDIGHUnqzyFLrRaGMcTkMNMqYo21%0AZv1zl6RRs4dMq4Vh9LyAvS+rhOW32usmaZVnINO0VpVjfoam7X2Z1VXee22W//lPs7pOSdKqzUCm%0AaY39MT9D6n0Z630qSbobA5lGbwi9TpIkrcoMZBo9j/mRJOkuPMtSkkZhCGeDeiawtPqwh0ySRmEI%0APcNg77C0urCHTJIkqWMGstXIMC6OKmmOG8JQ6DCGQb0/qDQ9hyxXI2N/iQppHK0iJ8n4/5M0PQOZ%0ApP68PMn48rOXRs5AJqm/VaTnRUPgZy+NnMeQSZIkdcwesq44JCBJkloGsq44JCBJkloOWUqSVk2r%0AyCU/NPtWx8uozLkesiQ7Ax8D1gA+V1Xv67gkSdJc5EjD2FodL6Myp3rIkqwB/F9gF2BrYI8kW3db%0AlSRJ42F17HlaVcy1HrLHARdW1UUASb4KPB84u9OqJEkaA6tjz9OqIlXVdQ13SPJiYOeq+rv2+Z7A%0A46vq9T2v2Q/Yr336COC8EZS2EXDVCLaju3K/d8P93g33++i5z7sxzvt9i6qa12/GXOshm1FVHQIc%0AMsptJjm5qhaNcptyv3fF/d4N9/vouc+74X7vb04dQwYsBjbveb5Z2yZJkrTammuB7CRgqyQPSXJv%0A4GXAMR3XJEmSNFRzasiyqm5L8nrgBzSXvfh8VZ3VcVkw4iFS3cH93g33ezfc76PnPu+G+72POXVQ%0AvyRJ0jiaa0OWkiRJY8dAJkmS1DED2TSS7JzkvCQXJnlr1/WMiyQXJzkjyWlJTu66ntVVks8nuTLJ%0AmT1tGyb5YZIL2p8P6LLG1dEU+/1dSRa33/nTkjynyxpXR0k2T/KTJGcnOSvJm9p2v/NDNM1+9zs/%0AiceQTaG9jdP5wLOAy2jOAN2jqrxrwJAluRhYVFXjeuHAkUjyFOBG4AtVtU3b9gHgmqp6X/tHyAOq%0A6sAu61zdTLHf3wXcWFUf6rK21VmS+cD8qjo1ybrAKcALgL3xOz800+z33fE7fxf2kE3tjts4VdWf%0AgYnbOEmrhar6KXDNpObnA4e304fT/MepWTTFfteQVdWSqjq1nb4BOAfYFL/zQzXNftckBrKpbQpc%0A2vP8MvwSjUoBxyU5pb1VlkZn46pa0k5fAWzcZTFj5vVJTm+HNB02G6IkC4HtgBPxOz8yk/Y7+J2/%0ACwOZ5qIdq2p7YBfgde0Qj0asmuMZPKZhND4NbAlsCywBPtxpNauxJOsARwP7V9X1vfP8zg9Pn/3u%0Ad34SA9nUvI1TR6pqcfvzSuCbNMPHGo2l7TEfE8d+XNlxPWOhqpZW1fKquh34LH7nhyLJWjSh4MtV%0A9Y222e/8kPXb737n785ANjVv49SBJPdvD/wkyf2BZwNnTr+UZtExwF7t9F7AtzusZWxMBILWC/E7%0AP+uSBDgUOKeqPtIzy+/8EE213/3O351nWU6jPQ33P7jzNk7v7bai1V+Sh9L0ikFza6+vuN+HI8kR%0AwE7ARsBS4J3At4CjgAXAJcDuVeUB6LNoiv2+E83QTQEXA6/uOa5JsyDJjsDPgDOA29vmg2iOZ/I7%0APyTT7Pc98Dt/FwYySZKkjjlkKUmS1DEDmSRJUscMZJIkSR0zkEmSJHXMQCZJktQxA5mkKSV5e5Kz%0A2tubnJbk8TO8/rAkL74H27tfkquTrDep/VtJXjrFMguTjPQaRknum+SEJGsk2SnJsZPmz7gfkuzW%0A3syaJC9IsvUA210ryant9PL2MzkryW+SHJBkpf5PT/KoJIetzLKSZoeBTFJfSZ4A7ApsX1V/BTyT%0Au97fddZV1R+BH9BcKHKijvWBHYHvDHPbK+hVwDeqavnKrqCqjqmq97VPXwDMGMho9sMv2umbq2rb%0Aqnok8CyaW429cyVrOQPYLMmClVle0j1nIJM0lfnAVVV1C0BVXVVVlwMk+eckJyU5M8kh7dW47yLJ%0AY9pepFOS/KDn9jRvTHJ22+v21T7bPYLmzhgTXkgT0m5O8sF2m2f06zFLsneST/Y8PzbJTu30je3y%0AZyX5UZLHJTk+yUVJdmtfs0b7mpPa+l49xb55OQNe0T3JxUn+Jcmpbd1/0VtrkicCuwEfbHu8tpxm%0AH+0MfH/yNtrbjO1Hc7PmtL2GP2u3eWq7DZJ8IckLemr7cpLnt0+/w133u6QRMpBJmspxwOZJzk/y%0AqSRP7Zn3yap6bFVtA9yXpiftDu296z4BvLiqHgN8Hpi448Jbge3aXrfX9NnuD4Dtkzywff4ympD2%0AIporez+aprfug5NuvzKT+wP/3fYo3QD8K03P0guBd7ev2Re4rqoeCzwW+D9JHjLpvd0beGhVXbwC%0A276qqranuaHyP/TOqKr/obl9zz+2PV6/Zep99DTg+H4bqKqLaO4q8iCa+zE+q93mS4GPty87FNi7%0AfR/rA08EvtvOOxl48gq8J0mzyEAmqa+quhF4DE3PyzLgyCR7t7OfluTEJGcATwceOWnxRwDbAD9M%0AchpwMLBZO+904MtJXgHc1me7f6YJKC9OshGwHU1I2xE4or0h8VLgBJrQNKg/A//VTp8BnFBVt7bT%0AC9v2ZwOvbGs+EXggsNWk9WwEXNtb8hTb622fuJH1KT3bms7d9lGSTYFr2mHdmawFfLb9fL5GOxxa%0AVSfQ3KN3Hs2ta46uqonP4EpgkwHWLWkI1uy6AElzV3uM1PHA8e0v973aIbRPAYuq6tIk7wLWnrRo%0AgLOq6gl9Vvtc4CnA84C3J3lUTyiYcATwjnY9366qW/uMivZzG3f9Q7O3rlvrznvF3Q5MDMXenmTi%0A/8IAb6iqH0yzjZsnrfdq4AGTXrMhcFXP81van8sZ7P/du+0jmuHKKetKcx/Y5TTB6p0098l8NM3+%0A+FPPS78AvIKm53Gfnva1ad6bpA7YQyaprySPSNLbO7Qtzc2XJ8LIVUnWAfqdTXgeMK89MWDi7MBH%0AtmcBbl5VPwEOBNYH1umz/PE0PVOvowln0Nyg+KXtcV7zaALLryYtdzGwbZJ7JdkceNwKvGVoAs9r%0A2yFXkjw8yf17X1BVfwDWSDKxHy4ANknyl+0yW9AEodNWYLs3AOu2y0+1j/oeP9YuMw/4DM1QcrXL%0ALKmq24E9aYYyJxwG7N++l7N72h8OjPRsVUl3sodM0lTWAT6RZAOanqcLgf2q6tokn6X55X0FcNLk%0ABavqz2ku+/Dx9lilNYH/AM4HvtS2Bfh4VV3bZ/nbk3wd2J1maBLgm8ATgN/QDAf+U1VdkWRhz6K/%0AAH4HnA2cA5y6gu/5czRDiqe2JyosozkDcrLjaIZQf1RVt7RDi//ZhrRbgb+rqutWYLtfpRlifCNN%0Az9WhvfuIJrA9rKrO7Vnmvu3Q6lo0n88XgY+08z4FHJ3klTTDtDdNLFRVS5OcA3xrUg1P487jySSN%0AWO7swZckDSLJ9sCbq2rPEW1vR+AVVdXvJIgVXdf9aI6b234iNCa5D03w3bHP8LGkEXDIUpJWUFWd%0ACvwkyRozvnh2tvfzWQpjz6TpOfzEpB68BcBbDWNSd+whkyRJ6pg9ZJIkSR0zkEmSJHXMQCZJktQx%0AA5kkSVLHDGSSJEkd+/+sUoM+agvw5QAAAABJRU5ErkJggg==%0A)

### Distribution for Important Products with low variability<a href="#Distribution-for-Important-Products-with-low-variability" class="anchor-link">¶</a>

In \[19\]:

    df_abc[(df_abc['CV']<0.3) & (df_abc['TO%']>0.25)]

Out\[19\]:

|     | item_id        | dept_id   | cat_id  | units | TO       | mean      | std      | CV       | Normality_p | Not_Normal | TO%      | TO%\_CS   | SKU_ID | SKU\_%   | ABC |
|-----|----------------|-----------|---------|-------|----------|-----------|----------|----------|-------------|------------|----------|-----------|--------|----------|-----|
| 9   | HOBBIES_1\_323 | HOBBIES_1 | HOBBIES | 8270  | 40822.60 | 22.657534 | 6.291498 | 0.277678 | 0.021937    | False      | 1.424885 | 17.325631 | 10.0   | 3.154574 | A   |
| 14  | HOBBIES_1\_337 | HOBBIES_1 | HOBBIES | 9685  | 35543.95 | 26.534247 | 7.677715 | 0.289351 | 0.003680    | False      | 1.240638 | 23.978597 | 15.0   | 4.731861 | A   |

In \[20\]:

    # ABC SKU-LEVEL
    df_dist = df[df['cat_id']=='HOBBIES'].drop(['mean', 'std', 'CV'], axis = 1).copy()
    df_dist = pd.DataFrame(df_dist.groupby(['item_id', 'dept_id', 'cat_id']).sum())
    df_dist.reset_index(inplace = True)

    # Item A
    item_high = 'HOBBIES_1_323'
    df_dist = df_dist[df_dist['item_id']==item_high][COLS_DATE].T
    df_dist.columns = ['Units']

    # Simple histogram
    df_dist['Units'].hist(figsize=(10,7), edgecolor='black', grid = False, bins = df_dist['Units'].nunique()) # , linewidth=1.2 , bins = 30
    plt.xlabel('Sales Volume (Units/Day)')
    plt.ylabel('Number of Days per Year')
    plt.title('Distribution of Sales for item: {}'.format(item_high))
    plt.show()

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAl4AAAG5CAYAAABfiDohAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90%0AbGliIHZlcnNpb24zLjMuNCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8QVMy6AAAACXBIWXMAAAsT%0AAAALEwEAmpwYAAAp4UlEQVR4nO3deZwlVX338c+XAQVlU5ngwDCg4hJwQcTdREVNMIy4RKNGEQgJ%0Axpio0TyCBI1rIjGKW4wPEQPuIPooYgy4ACqJsrkCIoqOiKwqCAQFht/zR1XDpe3pvt3T9/T07c/7%0A9epX31t1T9Xv3prp/vY5p6pSVUiSJGn0NlroAiRJkpYKg5ckSVIjBi9JkqRGDF6SJEmNGLwkSZIa%0AMXhJkiQ1YvCSJElqxOClJSvJe5O8ep62tSrJdUmW9c9PTfLn87HtfnufS7LffG1vFvt9Y5Krklw2%0Az9t9XJKfzuc2++2+KMnl/bG42zxs79wkj1v/yiSpY/DSWEry4yQ3JLk2ydVJ/jvJXya59d98Vf1l%0AVb1hyG09cbrXVNVPqmrzqlo7D7W/NsmHJm3/yVV1zPpue5Z1rAJeAexSVXdfx2sOTfKjPuj8NMmx%0ALWucVMsmwNuAP+iPxc/Xd5tVtWtVndpv/7eOS0tThfmpAmyS1UnOSHJ9kp8n+XCSlQPr90+ytj9m%0A1yW5KMmLBtbvlKQG1l+e5D395zvxmlv/T0yxvYmv7fr1j+n//12T5BdJTk/y0Bne64okJyT5WV/L%0ATkN+Rv+c5OIkv0qyJsmhA+vuk+TTSa7s6zgpyX0H1j8nyQV9nVckOSbJlsPsV5oNg5fG2VOqagtg%0AR+DNwMHAUfO9kyQbz/c2NxCrgJ9X1RVTrex74PYFnlhVmwN7AF9sWN9k2wKbAufOtmE6i/7nYZJn%0AAh8B3g5sA+wK/Ab4apK7DLz0f/pwujnwx8A/J3nwpM1t3a9/APBI4MXT7PrW7Q18/awPLicC7wLu%0ACmwPvK6vaTq3AP/V1zYbRwH3q6otgUcBz0vyjIn3A5wA3Jfu38oZwKcH2p4OPLqqtgLuCWwMvHGW%0A+5dmtOh/0EgzqaprquoE4NnAfknuD5Dk6CRv7B9vk+TEvnfsF0m+kmSjJB+kCyCf6f+Kf+VAj8CB%0ASX4CfGlg2WAIu1ff8/Cr/i/tu/b7mqqX4sdJnphkL+BQ4Nn9/r7Vr7+1t6Ov67D+L/orknwgyVb9%0Auok69kvyk3TDhH+/rs8myVZ9+yv77R3Wb/+JwOeB7fo6jp6i+UOBk6rqh/3nfFlVHTmw7QOSnJ+u%0A1/GiJC+cpo7tknyir+NHSV4ysO5hSc7qP8fLk7xtivb3AS7on16d5Ev98kclObPvxTgzyaMG2pya%0A5E1JTgf+l+6X7eTtznRctkpyVJJLk1ySbmh2Yrh5/3S9O0f0/64u6uvZP12vzBWZx+HjJAHeCryx%0Aqj5SVTdU1WXAnwPXAX87Vbuq+gZwPvC761h/Bd2/hV3mUNZ9+m18tKrW9jWdXFXfnq5RVV1eVe8B%0AzpzNzqrqgqq6fmDRLcDO/bozquqoqvpFVd0EHAHcN/2QdFVdXFVXDbRdO9FWmk8GLy0ZVXUG8FPg%0A96ZY/Yp+3XK6v4YP7ZrUvsBP6HrPNq+qfx5o81i6X1Z/uI5dvgD4M2AFcDPwziFq/C/gH4Fj+/09%0AaIqX7d9/PZ4uLGwOvHvSax5D95f9E4DXJJnylypdT8TEX/iP7Ws+oKq+ADwZ+Flfx/5TtP0a8IIk%0A/yfJHhOBY8AVwGpgS+AA4Igku0/eSN/T9BngW3Q9Ik8AXpZk4nN9B/COvhfjXsBxk7dRVd+n692B%0Arqdmzz7ofpbuc78b3TDkZ3P7uV/7AgcBWwBr1vEZTXdcjqY7tjsDDwb+gC7oTHg48O1+/x8BPkYX%0AWHcGng+8O8nm/efwp0mmDSQzuC/dHwkfn1T7LcAngCdN1SjdsN99gLPWsX47un/jX5tDTd8H1qYb%0AtnvypF63kUhySJLr6P4/35nuc5/K7wOXDQ5JpxsWvQa4lq637e0jLldLkMFLS83P6IY8JruJLiDt%0AWFU3VdVXauY7yL+2qq6vqhvWsf6DVfXd/i/wVwN/MkU4mYvnAW+rqouq6jrgVcBzJvW2va7vXfgW%0AXaD5rQDX1/Ic4FVVdW1V/Ziux2TfYYqoqg8Bf0P3S/k04IokBw+s/2xV/bA6pwEnM3XofSiwvKpe%0AX1U3VtVFwL/3tUF3bHZOsk1VXVdVwwaAvYELq+qDVXVzVX0U+B7wlIHXHF1V5/brbxpyuwAk2Rb4%0AI+Bl/b+DK+h6UZ4z8LIfVdV/9HP/jgV2AF5fVb+pqpOBG7mtR+YjVfXAGXb7zr737OokV9MN403Y%0Apv9+6RTtLh1YD/CIfhvX0g25fRC4cFKbq/p9XAJcDxw/TV2PGKwryUQv6K/o/ggoumN6Zbq5W9vO%0A8D7nrKreTBekd6d7X9dMfk26OW//Crx8Utuv9kONK4G3AD8eVZ1augxeWmq2B34xxfK3AD8ATu6H%0AhA4ZYlsXz2L9GmATbv/Lb6624/a9M2vo5qMM/jIbPAvxf+l6xSbbpq9p8ra2H7aQqvpwVT2Rbv7M%0AXwJvmOip6ns4vpZu6PZqupAy1fvfkW5IczBQHDrwfg6k65H5Xj9cuHrI8iZ/TlO9v5mO4XR2pPv8%0ALh2o+/8CvzPwmssHHt8A3TDapGVTHZt1eUlVbT3xRdejOGFimGzFFO1WDKwH+Fq/jS2Au9P1Fv7j%0ApDbb9Pu4E938p5Omqetrg3VV1b0mVlTV+VW1f1WtBO5Pd1zePtMbXR992P8G3ef7usF1SZbT/RHw%0Anj6MT9X+Ero5Zh8bZZ1amgxeWjL6IZXtga9OXtf3+Lyiqu4J7AO8PMkTJlavY5Mz9YjtMPB4FV3P%0AzVV0vQd3GqhrGd0Q57Db/RndL/3Bbd/M7X/JD+OqvqbJ27pkltuh7yX8ON2w2v2T3JFueOtfgG37%0AX+D/CWSK5hfT9QwN/uLeoqr+qN/2hVX1XLpAczhwfJI7D1HW5M9pqvc302c9aPJrL6abJL7NQN1b%0AVtWuU7Rt4QK64bVnDS7sh3L/mHWc+NAHwU9w+57AwfU30A2pPiLJev3hUFXf67d1//XZzixsTDc8%0ADUA/1HkycEJVvWk2baX5YvDS2EuyZd9L8jHgQ1X1nSleszrJzv0E5WvoJtbe0q++nCkmXg/h+Ul2%0ASXIn4PXA8f2Q0/eBTZPsne4U/cOAOw60uxzYKes+y+6jwN8muUc/P2hi7tHNsymur+U44E1Jtkiy%0AI93Qy1CXTOgnie/dt90oyZPpek6+Dtyhf09XAjf36/5gHZs6A7g2ycFJNkuyLMn9+6BMkucnWd7P%0AVbq6b3PLOrY16D+B+/RzpzZO8my6CeInztBuXW53XKrqUrpf4m/t/41tlOReSR47x+2vl35o/O+A%0Aw/r3vGmSuwPvo5tnd8RU7fo5b09nHWeD9iF6X7pe1FldoiPJ/ZK8oh/aI8kOwHMZYr5Ykk257f/F%0AHfvn071+oyQvTHKXdB5GdybmF/v1W9L12p1eVb/Vo53keekuoUL/f+FNLOxZuhpTBi+Ns8/0c1gu%0ABv6ebnL1Aet47b2BL9Cd/fU/dMMQp/Tr/onul9nVSf5uFvv/IN1f95fRXebgJdCdZQn8Fd0vxIn5%0AM4NnOU5Mjv55knOm2O77+21/GfgR8Gu6uVZz8Tf9/i+i6wn8SL/9YfyKbkjwJ3SB6J+BF/XzZK6l%0Ae7/HAb8E/pTuVP7f0gfA1cBudO/nKrrPZqv+JXsB5/YTpt8BPGeaeXWD2/15v91X0AWGVwKr6/Zn%0Ars3GVMflBXQh8zy693k8Uw/1zaj/xT/rS2EMqqpj6ULS39K95/OAzegukzAYmh6Z/npbdGc0Xslv%0A/xu6ul9/Od3lJPaZZt7jrdsb+Hoo3ST1hwNfT3I9XeD6Lt0xmckNdP8foZubN+MxpwuQP+z3+yG6%0Ak0feNbDuocABk+pc1a/fBfjvvs7T6XoQ/2KIfUqzkpnnD0uSJGk+2OMlSZLUiMFLktRcunulTh6e%0AvC7Je2doN1Wb65JMdakSaYPjUKMkSVIji+Iec9tss03ttNNOC12GJEnSjM4+++yrqmr5VOsWRfDa%0AaaedOOusKe9mIUmStEFJss5bkDnHS5IkqRGDlyRJUiMGL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi%0A8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JkqRGDF6SJEmNGLwkSZIaMXhJkiQ1YvCSJElqxOAl%0ASZLUiMFLkhpbsXIVSeb0tWLlqoUuX9J62HihC5CkpeaySy5mx4NPnFPbNYevnudqJLVkj5ckSVIj%0ABi9JkqRGDF6SJEmNGLwkSZIaMXhJkiQ1YvCSJElqxOAlSZLUiMFLkiSpEYOXJElSIwYvSZKkRgxe%0AkiRJjRi8JEmSGjF4SZIkNWLwkiRJasTgJUmS1IjBS5IkqRGDlyRJUiMGL0mSpEYMXpIkSY0YvCRJ%0AkhoxeEmSJDVi8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JkqRGDF6SJEmNGLwkSZIaMXhJkiQ1%0AYvCSJElqxOAlSZLUiMFLkiSpEYOXJElSIwYvSZKkRgxekiRJjRi8JEmSGjF4SZIkNWLwkiRJasTg%0AJUmS1IjBS5IkqRGDlyRJUiMGL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi8JIkSWrE4CVJktSIwUuS%0AJKkRg5ckSVIjIw9eSZYl+UaSE/vn90jy9SQ/SHJskjuMugZJkqQNQYser5cC5w88Pxw4oqp2Bn4J%0AHNigBkmSpAU30uCVZCWwN/C+/nmAPYHj+5ccAzxtlDVIkiRtKEbd4/V24JXALf3zuwFXV9XN/fOf%0AAttP1TDJQUnOSnLWlVdeOeIyJUmSRm9kwSvJauCKqjp7Lu2r6siq2qOq9li+fPk8VydJktTexiPc%0A9qOBfZL8EbApsCXwDmDrJBv3vV4rgUtGWIMkSdIGY2Q9XlX1qqpaWVU7Ac8BvlRVzwNOAZ7Zv2w/%0A4NOjqkGSJGlDshDX8ToYeHmSH9DN+TpqAWqQJElqbpRDjbeqqlOBU/vHFwEPa7FfSZKkDYlXrpck%0ASWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JkqRGDF6SJEmNGLwkSZIaMXhJkiQ1YvCSJElqxOAlSZLU%0AiMFLkiSpEYOXJElSIwYvSZKkRgxekiRJjRi8JEmSGjF4SZIkNWLwkiRJasTgJUmS1IjBS5IkqRGD%0AlyRJUiMGL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi8JIkSWrE4CVpXqxYuYokc/pasXLVQpcvSU1s%0AvNAFSBoPl11yMTsefOKc2q45fPU8VyNJGyZ7vCRJkhoxeEmSJDVi8JIkSWrE4CVJktSIwUuSJKkR%0Ag5ckSVIjBi9JkqRGDF6SJEmNGLwkSZIaMXhJkiQ1YvCSJElqxOAlSZLUiMFLkiSpEYOXJElSIwYv%0ASZKkRgxekiRJjRi8JEmSGjF4SZIkNWLwkiRJasTgJUmS1IjBS5IkqRGDlyRJUiMGL0mSpEYMXpIk%0ASY0YvCRJkhqZNnglWZbkX1oVI0mSNM6mDV5VtRZ4TKNaJEmSxtrGQ7zmG0lOAD4OXD+xsKo+ObKq%0AJEmSxtAwwWtT4OfAngPLCjB4SZIkzcKMwauqDmhRiCRJ0ribMXgl2RQ4ENiVrvcLgKr6sxHWJUmS%0ANHaGuZzEB4G7A38InAasBK4dZVGSJEnjaJjgtXNVvRq4vqqOAfYGHj7asiRJksbPMMHrpv771Unu%0AD2wF/M7oSpIkSRpPw5zVeGSSuwCvBk4ANgdeM9KqJEmSxtAwZzW+r394GnDP0ZYjSZI0vmYcakyy%0AbZKjknyuf75LkgNHX5okSdJ4GWaO19HAScB2/fPvAy8bUT2SJElja5jgtU1VHQfcAlBVNwNrR1qV%0AJEnSGBomeF2f5G50twkiySOAa0ZalSRJ0hha5+T6JM+uqmOBl9OdzXivJKcDy4FnNqpPkiRpbEzX%0A47Vvkv8CrgYeCzwKeCGwa1V9e6YNJ9k0yRlJvpXk3CSv65ffI8nXk/wgybFJ7jAP70OSJGmDt87g%0AVVWrgfcCnwVeBVwO/AzYIsldh9j2b4A9q+pBwG7AXv0w5eHAEVW1M/BLuvtASpIkjb1pr+NVVZ9K%0A8iPgy3QBqSZWMcM1vaqqgOv6p5v0XwXsCfxpv/wY4LXAv82hdkmSpEVlujledwQOo5vP9byqOnG2%0AG0+yDDgb2Bn4V+CHwNX9mZEAPwW2X0fbg4CDAFatWjXbXUuSJG1wppvj9W1gGbD7XEIXQFWtrard%0AgJXAw4D7zaLtkVW1R1XtsXz58rnsXpIkaYMy3VDj06vqvPnYSVVdneQU4JHA1kk27nu9VgKXzMc+%0AJEmSNnTTTa5fr9CVZHmSrfvHmwFPAs4HTuG2y1HsB3x6ffYjSZK0WMx4k+z1sAI4pp/ntRFwXFWd%0AmOQ84GNJ3gh8AzhqhDVIkiRtMKYNXn1oeklVHTHbDffX+nrwFMsvopvvJUmStKRMe8ugqloLPLdR%0ALZIkSWNtmKHG05O8GzgWuH5iYVWdM7KqJEmSxtAwwWu3/vvrB5ZNXAhVkiRJQ5oxeFXV41sUIkmS%0ANO6mneMFkGTbJEcl+Vz/fJck3l9RkiRplmYMXsDRwEnAdv3z7wMvG1E9kiRJY2uY4LVNVR0H3ALQ%0AX3F+7UirkiRJGkPDBK/rk9yNbkI9SR4BXDPSqiRJksbQMGc1vhw4AbhXktOB5dx2yx9JkiQNaZiz%0AGs9J8ljgvkCAC6rqppFXJkmSNGZmDF5JNgX+CngM3XDjV5K8t6p+PeriJC0RyzYhydya3mFT1t44%0Atx9Hd99+By796U/m1FaS5mKYocYPANcC7+qf/ynwQeBZoypK0hKz9iZ2PPjEOTVdc/jq9WorSS0N%0AE7zuX1W7DDw/Jcl5oypIkiRpXA1zVuM5/ZmMACR5OHDW6EqSJEkaT8P0eD0E+O8kExMhVgEXJPkO%0AUFX1wJFVJ0mSNEaGCV57jbwKSZKkJWCYy0msaVGIJEnSuBtmjpckSZLmgcFLkiSpkRmDV5I7J9mo%0Af3yfJPsk2WT0pUmSJI2XYXq8vgxsmmR74GRgX+DoURYlSZI0joYJXqmq/wWeAbynqp4F7DrasiRJ%0AksbPUMErySOB5wGf7ZctG11JkiRJ42mY4PVS4FXA/6uqc5PcEzhltGVJkiSNn2EuoPrLqtpn4klV%0AXQS8ZHQlSZIkjadherzek+SMJH+VZKuRVyRJrSzbhCRz+lqxctVCVy9pERrmyvW/l+Q+wAHA2UnO%0AAI6uqpNHXp0kjdLam9jx4BPn1HTN4avnuRhJS8FQF1Ctqu8DhwEHA48F3pHke0meMcriJEmSxskw%0AF1B9YJIjgPOBPYGnVNXv9o+PGHF9kiRJY2OYyfXvAt4HHFpVN0wsrKqfJTlsZJVJkiSNmWHmeD12%0AmnUfnN9yJEmSxteMwSvJvYF/AnYBNp1YXlX3HGFdkiRJY2eYyfX/AfwbcDPweOADwIdGWZQkSdI4%0AGiZ4bVZVX6S7Z+OaqnotsPdoy5IkSRo/w0yu/02SjYALk/w1cAmw+WjLkiRJGj/D3qvxTnS3CXoI%0AsC+w3yiLkiRJGkfDnNV4Zv/wOrqr10uSJGkOpu3xSrJfknOSXN9/nZXkBa2KkyRJGifr7PFKsh/w%0AMuDlwDlAgN2BtyQpr+ElSZI0O9P1eL0IeHpVnVJV11TV1VX1JeCPgRe3KU+SJGl8TBe8tqyqH09e%0A2C/bclQFSZIkjavpgtcNc1wnSdoArVi5iiSz/lqxctVCly6NjenOavzdJN+eYnkAbxckSYvMZZdc%0AzI4HnzjrdmsOXz2CaqSladrg1awKSZKkJWCdwauq1rQsRJIkadwNc+V6SZIkzQODlyRJUiPrDF5J%0Avth/P7xdOZIkSeNrusn1K5I8Ctgnycfozma8VVWdM9LKJEmSxsx0wes1wKuBlcDbJq0rYM9RFSVJ%0AkjSOpjur8Xjg+CSvrqo3NKxJkiRpLE3X4wVAVb0hyT7A7/eLTq2q2V+BT5IkaYmb8azGJP8EvBQ4%0Ar/96aZJ/HHVhkiRJ42bGHi9gb2C3qroFIMkxwDeAQ0dZmCRJ0rgZ9jpeWw883moEdUiSJI29YXq8%0A/gn4RpJT6C4p8fvAISOtSpIkaQzN2ONVVR8FHgF8EvgE8MiqOnbUhUmSlrYVK1eRZE5fK1auWujy%0ApSkN0+NFVV0KnDDiWiRJutVll1zMjgfP7ST6NYevnudqpPnhvRolSZIaMXhJkiQ1Mm3wSrIsyfda%0AFSNJkjTOpg1eVbUWuCCJsxQlSZLW0zCT6+8CnJvkDOD6iYVVtc/IqpIkSRpDwwSvV4+8CkmSpCVg%0AmJtkn5ZkR+DeVfWFJHcClo2+NEmSpPEyzE2y/wI4Hvi//aLtgU+NsCZJkqSxNMzlJF4MPBr4FUBV%0AXQj8ziiLkiRJGkfDBK/fVNWNE0+SbAzU6EqSJEkaT8MEr9OSHApsluRJwMeBz4y2LEmSpPEzTPA6%0ABLgS+A7wQuA/gcNmapRkhySnJDkvyblJXtovv2uSzye5sP9+l/V5A5IkSYvFMGc13pLkGODrdEOM%0AF1TVMEONNwOvqKpzkmwBnJ3k88D+wBer6s1JDqELdgfP+R1IkiQtEsOc1bg38EPgncC7gR8kefJM%0A7arq0qo6p398LXA+3RmRTwWO6V92DPC0OVUuSZK0yAxzAdW3Ao+vqh8AJLkX8Fngc8PuJMlOwIPp%0Aes22rapL+1WXAduuo81BwEEAq1Z5xyJJkrT4DTPH69qJ0NW7CLh22B0k2Rz4BPCyqvrV4Lp+yHLK%0AYcuqOrKq9qiqPZYvXz7s7iRJkjZY6+zxSvKM/uFZSf4TOI4uJD0LOHOYjSfZhC50fbiqPtkvvjzJ%0Aiqq6NMkK4Io5Vy9JkrSITDfU+JSBx5cDj+0fXwlsNtOGkwQ4Cji/qt42sOoEYD/gzf33T8+mYEmS%0ApMVqncGrqg5Yz20/GtgX+E6Sb/bLDqULXMclORBYA/zJeu5HkiRpUZhxcn2SewB/A+w0+Pqq2me6%0AdlX1VSDrWP2E4UuUJEkaD8Oc1fgpuiHDzwC3jLQaSZKkMTZM8Pp1Vb1z5JVIkiSNuWGC1zuS/ANw%0AMvCbiYUTF0eVJEnScIYJXg+gmyS/J7cNNVb/XJIkSUMaJng9C7hnVd046mIkSZLG2TBXrv8usPWI%0A65AkSRp7w/R4bQ18L8mZ3H6O17SXk5AkSdLtDRO8/mHkVUiSJC0BMwavqjqtRSGSJEnjbpgr119L%0AdxYjwB2ATYDrq2rLURYmSZI0bobp8dpi4nF/4+unAo8YZVGSJEnjaJizGm9VnU8BfziaciRJksbX%0AMEONzxh4uhGwB/DrkVUkSZI0poY5q/EpA49vBn5MN9woSZKkWRhmjtcBLQqRJEkad+sMXkleM027%0Aqqo3jKAeSZKksTVdj9f1Uyy7M3AgcDfA4CVJkjQL6wxeVfXWicdJtgBeChwAfAx467raSVq8Vqxc%0AxWWXXLzQZUjS2Jp2jleSuwIvB54HHAPsXlW/bFGYpPYuu+Ridjz4xDm1XXP46nmuRpLGz3RzvN4C%0APAM4EnhAVV3XrCpJkqQxNN0FVF8BbAccBvwsya/6r2uT/KpNeZIkSeNjujles7qqvSRJkqZnuJIk%0ASWrE4CVJktSIwUuSNL1lm5BkTl8rVq5a6OqlDcow92qUJC1la2/yMiPSPLHHS5IkqRGDlyRJUiMG%0AL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JkqRGDF6S%0AJEmNGLwkSZIaMXhJkiQ1YvCSJElqxOAlSZLUiMFLkiSpEYOXJElSIwYvSZKkRgxekiRJjRi8JEmS%0AGjF4SZIkNbLxQhcgSZqFZZuQZKGrGN5iq1caMYOXJC0ma29ix4NPnFPTNYevnudihrDY6pVGzKFG%0ASZKkRgxekiRJjRi8JEmSGjF4SZIkNWLwkiRJasSzGiVpLrxMgqQ5MHhJ0lx4mQRJc+BQoyRJUiMG%0AL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JkqRGDF6S%0AJEmNjCx4JXl/kiuSfHdg2V2TfD7Jhf33u4xq/5IkSRuaUfZ4HQ3sNWnZIcAXq+rewBf755IkSUvC%0AyIJXVX0Z+MWkxU8FjukfHwM8bVT7lyRJ2tC0nuO1bVVd2j++DNh2XS9MclCSs5KcdeWVV7apTpI0%0AHpZtQpI5fa1YuWqhq9cY23ihdlxVlaSmWX8kcCTAHnvssc7XSZL0W9bexI4HnzinpmsOXz3PxUi3%0Aad3jdXmSFQD99ysa71+SJGnBtA5eJwD79Y/3Az7deP+SJEkLZpSXk/go8D/AfZP8NMmBwJuBJyW5%0AEHhi/1ySJGlJGNkcr6p67jpWPWFU+5QkSdqQeeV6SZKkRgxe0jRWrFy1IKekr89+N77jZnNuK2n9%0ALNTPDC0eC3Y5CWkxuOySixfklPT13a+n0UsLY6F+ZmjxsMdLkiSpEYOXJElSIwYvSZKkRgxekiRJ%0AjRi8JEmSGjF4SZIkNWLwkiRp0LJNFt218Lx+2OLhdbwkSRq09qZFdy0urx+2eNjjJUmS1IjBS5Ik%0AqRGDlyRJUiMGL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIj%0ABi9JkqRGDF6SJEmNGLwkSZIa2XihC5DG1rJNSLLQVUiSNiAGL2lU1t7EjgefOKemaw5fPc/FSJI2%0ABA41SpIkNWLwkiRJasTgJUmS1IjBS5IkqRGDlyRJUiMGL0mSpEYMXpIkSY0YvCRJkhoxeEmSJDVi%0A8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JkqRGDF6SJEmNGLwkSZIaMXhJkiQ1YvCSJElqxOAl%0ASZLUiMFLkiSpEYOXJElSIwYvLQkrVq4iyay/JEkbnrn+TF+xctVCl87GC12A1MJll1zMjgefOOt2%0Aaw5fPYJqJEnrYzH/TLfHS5IkqRGDlyRJUiMGL0mSpEYMXpIkSY0YvCRJkhrxrEZJkjYEyzZZmMvY%0ArMd+l91hU9be+Os5tb379jtw6U9/Mqe2i5nBS5KkDcHam+Z0iQRYz8skrOd+F6TmRcyhRkmSpEYM%0AXpIkSY0YvCRJkhoxeEmSJDVi8JIkSWrE4CVJktSIwau3YuUqkszpa8XKVQtdflML9Vmtz34lSdoQ%0AeB2v3mWXXOy1SIa0UJ+Vx0iStNjZ4yVJktSIwUuSJKkRg5ckSVIjCxK8kuyV5IIkP0hyyELUIEmS%0A1Frz4JVkGfCvwJOBXYDnJtmldR2SJEmtLUSP18OAH1TVRVV1I/Ax4KkLUIckSVJTqaq2O0yeCexV%0AVX/eP98XeHhV/fWk1x0EHNQ/vS9wwYhL2wa4asT70IbD4720eLyXDo/10rKhHu8dq2r5VCs22Ot4%0AVdWRwJGt9pfkrKrao9X+tLA83kuLx3vp8FgvLYvxeC/EUOMlwA4Dz1f2yyRJksbaQgSvM4F7J7lH%0AkjsAzwFOWIA6JEmSmmo+1FhVNyf5a+AkYBnw/qo6t3UdU2g2rKkNgsd7afF4Lx0e66Vl0R3v5pPr%0AJUmSliqvXC9JktSIwUuSJKkRgxfewmjcJXl/kiuSfHdg2V2TfD7Jhf33uyxkjZofSXZIckqS85Kc%0Am+Sl/XKP9xhKsmmSM5J8qz/er+uX3yPJ1/uf6cf2J3JpDCRZluQbSU7sny+6Y73kg5e3MFoSjgb2%0AmrTsEOCLVXVv4Iv9cy1+NwOvqKpdgEcAL+7/P3u8x9NvgD2r6kHAbsBeSR4BHA4cUVU7A78EDly4%0AEjXPXgqcP/B80R3rJR+88BZGY6+qvgz8YtLipwLH9I+PAZ7WsiaNRlVdWlXn9I+vpfsBvT0e77FU%0Anev6p5v0XwXsCRzfL/d4j4kkK4G9gff1z8MiPNYGr+6H8sUDz3/aL9N427aqLu0fXwZsu5DFaP4l%0A2Ql4MPB1PN5jqx96+iZwBfB54IfA1VV1c/8Sf6aPj7cDrwRu6Z/fjUV4rA1eWvKqu6aK11UZI0k2%0ABz4BvKyqfjW4zuM9XqpqbVXtRncXlIcB91vYijQKSVYDV1TV2Qtdy/raYO/V2JC3MFqaLk+yoqou%0ATbKC7q9ljYEkm9CFrg9X1Sf7xR7vMVdVVyc5BXgksHWSjfueEH+mj4dHA/sk+SNgU2BL4B0swmNt%0Aj5e3MFqqTgD26x/vB3x6AWvRPOnnfBwFnF9VbxtY5fEeQ0mWJ9m6f7wZ8CS6eX2nAM/sX+bxHgNV%0A9aqqWllVO9H9nv5SVT2PRXisvXI90Cfot3PbLYzetLAVaT4l+SjwOGAb4HLgH4BPAccBq4A1wJ9U%0A1eQJ+FpkkjwG+ArwHW6bB3Io3Twvj/eYSfJAugnVy+g6Eo6rqtcnuSfdiVJ3Bb4BPL+qfrNwlWo+%0AJXkc8HdVtXoxHmuDlyRJUiMONUqSJDVi8JIkSWrE4CVJktSIwUuSJKkRg5ckSVIjBi9JJPn7JOcm%0A+XaSbyZ5+AyvPzrJM6d7zQzt75Tk50m2nLT8U0mevY42OyX57lz3ORdJNktyWn9bmsclOXHS+hk/%0AhyT7JDmkf/y0/qbdM+13kyTn9I/X9sfk3CTfSvKKJHP62Z3kAUmOnktbSfPD4CUtcUkeCawGdq+q%0ABwJP5Pb3L513VfW/wEnA0wfq2Ap4DPCZUe57lv4M+GRVrZ3rBqrqhKp6c//0acCMwYvuczi9f3xD%0AVe1WVbvSXSD0yXTXoptLLd8BViZZNZf2ktafwUvSCuCqiYsOVtVVVfUzgCSvSXJmku8mObK/Mvzt%0AJHlI3yt0dpKT+lvykOQlSc7re9E+NsV+P0p3BeoJT6cLYzckeUu/z+9M1QOWZP8k7x54fmJ/UUWS%0AXNe3PzfJF5I8LMmpSS5Ksk//mmX9a87s63vhOj6b5zHklbCT/DjJ65Kc09d9v8FakzwK2Ad4S9+D%0Ada9pPqO9gM9N3kdVXQEcBPx1Ojsl+Uq/z3P6fZDkA0meNlDbh5M8tX/6GW7/uUtqyOAl6WRghyTf%0AT/KeJI8dWPfuqnpoVd0f2IyuZ+xW6e6L+C7gmVX1EOD9wMSdHw4BHtz3ov3lFPs9Cdg9yd3658+h%0AC2PPAHYDHkTX+/aWiTA3pDvT3U5kV+Ba4I10PUVPB17fv+ZA4JqqeijwUOAvktxj0nu7A3DPqvrx%0ALPZ9VVXtDvwb8HeDK6rqv+luXfR/+h6sH7Luz+jxwKlT7aCqLqK7Uvvv0N1z8kn9Pp8NvLN/2VHA%0A/v372Ap4FPDZft1ZwO/N4j1JmkcGL2mJq6rrgIfQ9aRcCRybZP9+9eOTfD3Jd4A9gV0nNb8vcH/g%0A80m+CRxGd6NagG8DH07yfODmKfZ7I10QeWaSbYAH04WxxwAfraq1VXU5cBpdOBrWjcB/9Y+/A5xW%0AVTf1j3fql/8B8IK+5q8DdwPuPWk72wBXD5a8jv0NLp+4KffZA/uazm99Rkm2B37RD8fOZBPg3/vj%0A83H6YcyqOo3uHrTLgecCn+hvIgxdWNtuiG1LGoGNF7oASQuvn8N0KnBq/0t8v37o6z3AHlV1cZLX%0AAptOahrg3Kp65BSb3Rv4feApwN8necDAL/8JHwVe3W/n01V10xSjmVO5mdv/4ThY1011273QbgEm%0AhlBvSTLxMy/A31TVSdPs44ZJ2/05cJdJr7krcNXA84l7xK1luJ+vv/UZ0Q0zrrOudPemW0sXoP6B%0A7v6jD6L7PH498NIPAM+n60k8YGD5pnTvTdICsMdLWuKS3DfJYG/PbnQ3kp4IHVcl2RyY6uy9C4Dl%0A/QT9ibPxdu3Putuhqk4BDga2Ajafov2pdD1NL6YLYdDd5PrZ/Tys5XTB5IxJ7X4M7JZkoyQ7AA+b%0AxVuGLti8qB8qJcl9ktx58AVV9UtgWZKJz+FCYLskv9u32ZEu8HxzFvu9Ftiib7+uz2jK+V19m+XA%0Ae+mGgKtvc2lV3QLsSzcEOeFo4GX9ezlvYPl9gKZnh0q6jT1ekjYH3pVka7qepB8AB1XV1Un+ne6X%0A9GXAmZMbVtWN6S6n8M5+LtHGwNuB7wMf6pcFeGdVXT1F+1uSHA/8Cd2QIsD/Ax4JfItuGO+VVXVZ%0Akp0Gmp4O/Ag4DzgfOGeW7/l9dEOB5/QnDFxJd8bhZCfTDX1+oap+0w8J/kcfxm4C/ryqrpnFfj9G%0ANzT4ErqeqKMGPyO6YLZzVX1voM1m/ZDoJnTH54PA2/p17wE+keQFdMOr1080qqrLk5wPfGpSDY/n%0AtvlekhrLbT3ykqRBSXYH/raq9m20v8cAz6+qqU5GmO227kQ3r233iXCY5I50AfcxUwz7SmrAoUZJ%0AWoeqOgc4JcmyGV88P/v76jyFrifS9QS+a1KP3CrgEEOXtHDs8ZIkSWrEHi9JkqRGDF6SJEmNGLwk%0ASZIaMXhJkiQ1YvCSJElq5P8DgSieCuIsOqkAAAAASUVORK5CYII=%0A)

### Distribution for Low rotation Products with very high variability<a href="#Distribution-for-Low-rotation--Products-with-very-high-variability" class="anchor-link">¶</a>

\| Example of item with very high COV (>2)

In \[22\]:

    # ABC SKU-LEVEL
    df_dist = df[df['cat_id']=='HOBBIES'].drop(['mean', 'std', 'CV'], axis = 1).copy()
    df_dist = pd.DataFrame(df_dist.groupby(['item_id', 'dept_id', 'cat_id']).sum())
    df_dist.reset_index(inplace = True)

    # Item A
    item_high = 'HOBBIES_1_042'
    df_dist = df_dist[df_dist['item_id']==item_high][COLS_DATE].T
    df_dist.columns = ['Units']

    # Simple histogram
    df_dist['Units'].hist(figsize=(10,7), edgecolor='black', grid = False, bins = df_dist['Units'].nunique()) # , linewidth=1.2 , bins = 30
    plt.xlabel('Sales Volume (Units/Day)')
    plt.ylabel('Number of Days per Year')
    plt.title('Distribution of Sales for item: {}'.format(item_high))
    plt.show()

![](data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAmQAAAG5CAYAAAAgWSjQAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90%0AbGliIHZlcnNpb24zLjMuNCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8QVMy6AAAACXBIWXMAAAsT%0AAAALEwEAmpwYAAAuSUlEQVR4nO3dd7hlZX238fvrgILSVEakOopoAiQiGRWjiYIlqAhqLBilhYTE%0AYCeJSOzGxMQosUR9iRiwgQQbokYUEUsiMKLSUaRIZ0BAQETK7/1jPQe2x1P2DLP3Osy+P9d1rrP2%0AKs/6rb1nzvme51klVYUkSZL6c6++C5AkSZp0BjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKk%0AnhnIJEmSemYgk2aQ5MNJ3riK2toiyY1JFrXX30zyF6ui7dbeV5LstaraW4H9/mOSq5NcsYrbfXKS%0AS1Zlm63dlyW5sn0WD1wF7Z2Z5Ml3vzJJMpBpAiW5MMnNSW5Icl2S/03y10nu/P9QVX9dVW8fsq2n%0AzrVOVf2sqtapqttXQe1vSfKJae0/o6oOv7ttr2AdWwAHAFtX1YNnWeegJBe0AHRJkk+Ps8ZptawJ%0AvAd4evssrrm7bVbVNlX1zdb+b30u4zRTyJ8p2CbZJcnJSW5Kck2STybZbGD53klub5/ZjUnOT/Ky%0AgeVLktTA8iuTfLC9v1Pr3Pl/Yob2pr42acuf2P7/XZ/k50m+m+Qx8xzrxkmOSXJZq2XJkO/RfZJ8%0ANMkvklyR5LWzrPem1u5TB+b9W5KftJ8Z5yTZc5h9SivCQKZJ9eyqWhd4CPBO4HXAoat6J0nWWNVt%0ALhBbANdU1VUzLWw9dnsAT62qdYClwPFjrG+6jYC1gDNXdMN07vE/K5M8H/gU8O/AhsA2wC3Ad5Lc%0Af2DV/2uhdR3gT4F/TfLoac1t0Jb/HvB4YP85dn1newNflyVZDzgWeD/wAGBT4K2tprncAfxPq21F%0AvAXYiu7//I7A3yfZeXCFJFsCLwAun7btTcCzgfWBvYD3JvnDFdy/NKd7/A8Z6e6oquur6hjgRcBe%0ASbYFSHJYkn9s0xsmObb1pv08ybeT3CvJx+mCyRfbX/1/P9CDsG+SnwHfGJg3GM62bD0Vv0jyhSQP%0AaPuaqVfjwiRPbb88DgJe1Pb3o7b8zt6RVtcbklyU5KokH0uyfls2VcdeSX6WbrjxH2Z7b5Ks37Zf%0A3tp7Q2v/qcDXgE1aHYfNsPljgK9W1U/b+3xFVR0y0PY+Sc5uPQ7nJ/mrOerYJMlnWh0XJHnlwLLH%0AJlnW3scrk7xnhu0fAZzbXl6X5Btt/h8mOaX1zpwy+Au2vafvSPJd4JfAw2Zod77PZf0khya5PMml%0A6YZ4p4at907XG3Rw+3d1fqtn7yQXt89ulQ1DJwnwbuAfq+pTVXVzVV0B/AVwI/Cambarqh8AZwO/%0AO8vyq+j+LWy9EmU9orVxRFXd3mo6rqpOm2ujqrqyqj4InLKC+9sLeHtVXVtVZwP/Cew9bZ3/oPvj%0A7NfT9vnmqjqnqu6oqpOAb9MFUWmVMZBJQFWdDFwC/NEMiw9oyxbT9bQc1G1SewA/o+ttW6eq/nVg%0AmyfR/RL7k1l2uSfw58DGwG3A+4ao8X+AfwI+3fb3qBlW27t97UgXItYBPjBtnScCjwSeArwpyYy/%0AbOl6LtZv7Typ1bxPVX0deAZwWatj7xm2/R6wZ5K/S7J0KogMuArYBVgP2Ac4OMn20xtpPVNfBH5E%0A14PyFODVSabe1/cC762q9YAtgaOmt1FVP6brDYKuZ2enFoC/RPe+P5BuOPNL+c1zy/YA9gPWBS6a%0A5T2a63M5jO6zfTjwaODpdAFoyuOA09r+PwUcSRdkHw68FPhAknXa+/BnSeYMKvN4JN0fD/89rfY7%0AgM8AT5tpo3TDh48Als2yfBO6f+PfW4mafgzcnuTwJM/Ib/bSrVKt7Y3p/h1N+RF3/bsgyQuAW6rq%0Ay/O0tTbd57TCva3SXAxk0l0uoxs6me5Wuh/mD6mqW6vq21VV87T1lqq6qapunmX5x6vqjKq6CXgj%0A8MIZQsvKeAnwnqo6v6puBF4P7D6td+6trTfiR3S/lH4r2LVadgdeX1U3VNWFdD0sewxTRFV9AngF%0A3S/rE4GrkrxuYPmXquqn1TkROI6Zw/BjgMVV9baq+nVVnU/Xs7F7W34r8PAkG1bVjVU1bDB4FvCT%0Aqvp4Vd1WVUcA59ANS005rKrObMtvHbJdAJJsBDwTeHX7d3AVcPBA3QAXVNV/tXMLPw1sDrytqm6p%0AquPoemkeDtB6tX5/nt2+r/W2XZfkOrrhwCkbtu/Th+Km5m048HqH1sYNwMnAx4GfTNvm6raPS+mG%0A846eo64dButKMtVr+gu6Pw6K7jNdnu7csI3mOc6VsU77fv3AvOvpwjZJ1qUL1a8aoq0P0/2/+eqq%0ALFAykEl32RT4+Qzz3wWcBxzXhpYOHKKti1dg+UXAmvzmL8WVtQm/2ZtzEbAGXc/elMGrIn/JXb+s%0ABm3Yapre1qbDFlJVn6yqpwIbAH8NvH2qZ6v1iHwv3RDwdXThZabjfwjd0Ohg0Dho4Hj2pevBOacN%0AO+4yZHnT36eZjm++z3AuD6F7/y4fqPv/AQ8aWOfKgemboRuOmzZvps9mNq+sqg2mvuh6IKdc3b5v%0APMN2Gw8sB/hea2Nd4MF0vUj/NG2bDds+7gt8l7nDyfcG66qqLacWVNXZVbV3VW0GbEv3ufz7fAe6%0AEm5s39cbmLcecEObfgvdH0kXztVIknfR1fnCIf4ok1aIgUzizqGZTYHvTF/WeogOqKqHAbsCr03y%0AlKnFszQ53w/rzQemt6Dr6bmarrfhvgN1LaIbKh223cvowsBg27fxm7/8h3F1q2l6W5euYDu0XsX/%0Aphue2zbJfeiGyf4N2Kj9Yv8ykBk2v5iuJ2nwF/q6VfXM1vZPqurFdEHnX4Cjk9xviLKmv08zHd+K%0A/MKdvu7FdCenbzhQ93pVtc0M247DuXTD7i8YnNmGhP+UWS64aAHxM/xmz+Hg8pvphmZ3SHK3/qCo%0AqnNaW9venXZmaftaup7Awd7gR3HXsONTgFemu/ryCrr/n0cN9uomeSvdUP3TW++etEoZyDTRkqzX%0AelWOBD5RVafPsM4uSR7eToy+Hrid7kov6ILOb53wPYSXJtk6yX2BtwFHt6GrHwNrJXlWulsJvAG4%0Az8B2VwJLMvtVf0cAr0ny0Hb+0dS5TbetSHGtlqOAdyRZN8lDgNcCQ93aoZ2c/qy27b2SPIOup+Uk%0A4N7tmJYDt7VlT5+lqZOBG5K8LsnaSRYl2bYFaJK8NMnidi7UdW2bO2Zpa9CXgUe0c7PWSPIiuhPT%0Aj51nu9n8xudSVZfTDcO+u/0bu1eSLZM8aSXbv1tab87fAm9ox7xWkgcDH6HrKTp4pu3aOXXPZZbz%0ApVq43oOu13WFbiWS5HeSHJB2240kmwMvZojz0ZKsxV3/L+7TXs/nY3THf/8kvwP8JV0AhC6QbQts%0A174uA/6K7iR/krwe+DO6q4bv9i1TpJkYyDSpvtjOkbkY+Ae6k7r3mWXdrYCv0w17/B/wwao6oS37%0AZ7of8tcl+dsV2P/H6X4ZXEF3O4ZXQnfVJ/A3dL8op87PGbzqcuqk7GuSnDpDux9tbX8LuAD4Fd25%0AXCvjFW3/59P1HH6qtT+MX9ANLf6MLij9K/CyqvpOVd1Ad7xHAdfS/aI7ZqZGWjDche6X5AV0PXcf%0AobvYAGBn4MwkN9Kd4L/7HOftDbZ7TWv3ALog8ffALlV19Zwbzm6mz2VPuvB5Ft1xHs3MQ4bzSvKS%0AJHfrJPKq+jRdeHoN3TGfBawNPGFayHh82v3C6K6wXM5v/xu6ri2/ku5qw13nGMK7s72Br8fQDRc+%0ADjgpyU10QewMus9kPjdz1zDkOe31fN4M/JRuaPpE4F3tggyq6prqrgS+orqrT28Hrm3nYUL3h80W%0AwHkDx3DQEPuUhhaHwSVJkvplD5kkSVLPDGSSpAUl3bNkpw9z3pjkw/NsN9M2NyaZ6ZYq0oLikKUk%0ASVLP7tHP2dtwww1ryZIlfZchSZI0r+9///tXV9XimZbdowPZkiVLWLZsxid6SJIkLShJZn0Mm+eQ%0ASZIk9cxAJkmS1DMDmSRJUs8MZJIkST0zkEmSJPXMQCZJktQzA5kkSVLPDGSSJEk9M5BJkiT1zEAm%0ASZLUMwOZJElSzwxkkiRJPTOQSZIk9WxkgSzJWklOTvKjJGcmeWubf1iSC5L8sH1t1+YnyfuSnJfk%0AtCTbj6o2SZKkhWSNEbZ9C7BTVd2YZE3gO0m+0pb9XVUdPW39ZwBbta/HAR9q3yVJklZrI+shq86N%0A7eWa7avm2GQ34GNtu+8BGyTZeFT1SZIkLRQjPYcsyaIkPwSuAr5WVSe1Re9ow5IHJ7lPm7cpcPHA%0A5pe0edPb3C/JsiTLli9fPsry77TxZluQ5B73tfFmW4zl/ZEkSXfPKIcsqarbge2SbAB8Lsm2wOuB%0AK4B7A4cArwPetgJtHtK2Y+nSpXP1uK0yV1x6MQ953bHj2NUqddG/7NJ3CZIkaQhjucqyqq4DTgB2%0ArqrL27DkLcB/AY9tq10KbD6w2WZtniRJ0mptlFdZLm49YyRZG3gacM7UeWFJAjwHOKNtcgywZ7va%0Acgfg+qq6fFT1SZIkLRSjHLLcGDg8ySK64HdUVR2b5BtJFgMBfgj8dVv/y8AzgfOAXwL7jLA2SZKk%0ABWNkgayqTgMePcP8nWZZv4D9R1WPJEnSQuWd+iVJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6%0AZiCTJEnqmYFMkiSpZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZ%0AgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcG%0AMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnI%0AJEmSemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCT%0AJEnqmYFMkiSpZyMLZEnWSnJykh8lOTPJW9v8hyY5Kcl5ST6d5N5t/n3a6/Pa8iWjqk2SJGkhGWUP%0A2S3ATlX1KGA7YOckOwD/AhxcVQ8HrgX2bevvC1zb5h/c1pMkSVrtjSyQVefG9nLN9lXATsDRbf7h%0AwHPa9G7tNW35U5JkVPVJkiQtFCM9hyzJoiQ/BK4Cvgb8FLiuqm5rq1wCbNqmNwUuBmjLrwceOEOb%0A+yVZlmTZ8uXLR1m+JEnSWIw0kFXV7VW1HbAZ8Fjgd1ZBm4dU1dKqWrp48eK725wkSVLvxnKVZVVd%0AB5wAPB7YIMkabdFmwKVt+lJgc4C2fH3gmnHUJ0mS1KdRXmW5OMkGbXpt4GnA2XTB7Plttb2AL7Tp%0AY9pr2vJvVFWNqj5JkqSFYo35V1lpGwOHJ1lEF/yOqqpjk5wFHJnkH4EfAIe29Q8FPp7kPODnwO4j%0ArE2SJGnBGFkgq6rTgEfPMP98uvPJps//FfCCUdUjSZK0UHmnfkmSpJ4ZyCRJknpmIJMkSeqZgUyS%0AJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcGMkmS%0ApJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnIJEmS%0AemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnq%0AmYFMkiSpZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKln%0ABjJJkqSeGcgkSZJ6ZiCTJEnq2cgCWZLNk5yQ5KwkZyZ5VZv/liSXJvlh+3rmwDavT3JeknOT/Mmo%0AapMkSVpI1hhh27cBB1TVqUnWBb6f5Gtt2cFV9W+DKyfZGtgd2AbYBPh6kkdU1e0jrFGSJKl3I+sh%0Aq6rLq+rUNn0DcDaw6Ryb7AYcWVW3VNUFwHnAY0dVnyRJ0kIxlnPIkiwBHg2c1Ga9PMlpST6a5P5t%0A3qbAxQObXcIMAS7JfkmWJVm2fPnyUZYtSZI0FiMPZEnWAT4DvLqqfgF8CNgS2A64HHj3irRXVYdU%0A1dKqWrp48eJVXa4kSdLYjTSQJVmTLox9sqo+C1BVV1bV7VV1B/Cf3DUseSmw+cDmm7V5kiRJq7VR%0AXmUZ4FDg7Kp6z8D8jQdWey5wRps+Btg9yX2SPBTYCjh5VPVJkiQtFKO8yvIJwB7A6Ul+2OYdBLw4%0AyXZAARcCfwVQVWcmOQo4i+4Kzf29wlKSJE2CkQWyqvoOkBkWfXmObd4BvGNUNUmSJC1E3qlfkiSp%0AZwYySZKkns0ZyJIsSvJvc60jSZKku2fOQNZOqn/imGqRJEmaSMOc1P+DJMcA/w3cNDVz6r5ikiRJ%0AunuGCWRrAdcAOw3MK8BAJkmStArMG8iqap9xFCJJkjSp5g1kSdYC9gW2oestA6Cq/nyEdUmSJE2M%0AYW578XHgwcCfACfSPWPyhlEWJUmSNEmGCWQPr6o3AjdV1eHAs4DHjbYsSZKkyTFMILu1fb8uybbA%0A+sCDRleSJEnSZBnmKstDktwfeCNwDLAO8KaRViVJkjRBhrnK8iNt8kTgYaMtR5IkafLMO2SZZKMk%0Ahyb5Snu9dZJ9R1+aJEnSZBjmHLLDgK8Cm7TXPwZePaJ6JEmSJs4wgWzDqjoKuAOgqm4Dbh9pVZIk%0ASRNkmEB2U5IH0j0uiSQ7ANePtCpJkqQJMutJ/UleVFWfBl5Ld3Xllkm+CywGnj+m+iRJklZ7c11l%0AuUeSfYC/AZ4EPBIIcG5V3TrHdpIkSVoBsw5ZVtUuwIeBLwGvB64ELgPWTfKA8ZQnSZK0+pvzPmRV%0A9fkkFwDfonvAeE0twnuSSZIkrRJznUN2H+ANdOeLvaSqjh1bVZIkSRNkrqssTwMWAdsbxiRJkkZn%0AriHL51bVWWOrRJIkaULNdVK/YUySJGkMhrkxrCRJkkZozkCWZFGS14yrGEmSpEk0ZyCrqtuBF4+p%0AFkmSpIk0533Imu8m+QDwaeCmqZlVderIqpIkSZogwwSy7dr3tw3MK2CnVV6NJEnSBJo3kFXVjuMo%0ARJIkaVLNe5Vlko2SHJrkK+311kn2HX1pkiRJk2GY214cBnwV2KS9/jHw6hHVI0mSNHGGCWQbVtVR%0AwB0AVXUbcPtIq5IkSZogwwSym5I8kO5EfpLsAFw/0qokSZImyDBXWb4WOAbYMsl3gcXA80dalSRJ%0A0gQZ5irLU5M8CXgkEODcqrp15JVJkiRNiHkDWZK1gL8Bnkg3bPntJB+uql+NujhJkqRJMMyQ5ceA%0AG4D3t9d/BnwceMGoipIkSZokwwSybatq64HXJyQ5a1QFSZIkTZphrrI8tV1ZCUCSxwHLRleSJEnS%0AZBmmh+wPgP9N8rP2egvg3CSnA1VVvz+y6iRJkibAMIFs55FXIUmSNMGGue3FRSvTcJLN6S4I2Iju%0A6sxDquq9SR4AfBpYAlwIvLCqrk0S4L3AM4FfAntX1akrs29JkqR7kmHOIVtZtwEHtAsCdgD2T7I1%0AcCBwfFVtBRzfXgM8A9iqfe0HfGiEtUmSJC0YIwtkVXX5VA9XVd0AnA1sCuwGHN5WOxx4TpveDfhY%0Adb4HbJBk41HVJ0mStFDMG8iS3C/Jvdr0I5LsmmTNFdlJkiXAo4GTgI2q6vK26Aq6IU3owtrFA5td%0A0uZNb2u/JMuSLFu+fPmKlCFJkrQgDdND9i1grSSbAscBewCHDbuDJOsAnwFeXVW/GFxWVUV7aPmw%0AquqQqlpaVUsXL168IptKkiQtSMMEslTVL4HnAR+sqhcA2wzTeOtJ+wzwyar6bJt95dRQZPt+VZt/%0AKbD5wOabtXmSJEmrtaECWZLHAy8BvtTmLRpmI+BQ4Oyqes/AomOAvdr0XsAXBubvmc4OwPUDQ5uS%0AJEmrrWHuQ/Yq4PXA56rqzCQPA04YYrsn0A1vnp7kh23eQcA7gaOS7AtcBLywLfsy3S0vzqO77cU+%0Awx6EJEnSPdkwgezaqtp16kVVnQ+8cr6Nquo7QGZZ/JQZ1i9g/yHqkSRJWq0MM2T5wSQnJ/mbJOuP%0AvCJJkqQJM28gq6o/Al5Kd8L995N8KsnTR16ZJEnShBjqxrBV9WPgDcDrgCcB701yTpLnjbI4SZKk%0ASTDMjWF/P8nBdHfa3wl4dlX9bps+eMT1SZIkrfaGOan//cBHgIOq6uapmVV1WZI3jKwySZKkCTFv%0AIKuqJ82x7OOrthxJkqTJM28gS7IV8M/A1sBaU/Or6mEjrEuSJGliDHNS/38BHwJuA3YEPgZ8YpRF%0ASZIkTZJhAtnaVXU83TMtL6qqtwDPGm1ZkiRJk2OYk/pvSXIv4CdJXk73wO91RluWJEnS5Bimh+xV%0AwH3pHpf0B3TPp9xrzi0kSZI0tGGusjylTd6ID/yWJEla5ebsIUuyV5JTk9zUvpYl2XNcxUmSJE2C%0AWXvIkuwFvBp4LXAqEGB74F1JynuQSZIkrRpz9ZC9DHhuVZ1QVddX1XVV9Q3gT4H9x1OeJEnS6m+u%0AQLZeVV04fWabt96oCpIkSZo0cwWym1dymSRJklbAXFdZ/m6S02aYH8DHJkmSJK0icwaysVUhSZI0%0AwWYNZFV10TgLkSRJmlTD3KlfkiRJI2QgkyRJ6tmsgSzJ8e37v4yvHEmSpMkz10n9Gyf5Q2DXJEfS%0AXV15p6o6daSVSZIkTYi5AtmbgDcCmwHvmbasgJ1GVZQkSdIkmesqy6OBo5O8sarePsaaJEmSJspc%0APWQAVNXbk+wK/HGb9c2qOna0ZUmSJE2Oea+yTPLPwKuAs9rXq5L806gLkyRJmhTz9pABzwK2q6o7%0AAJIcDvwAOGiUhUmSJE2KYe9DtsHA9PojqEOSJGliDdND9s/AD5KcQHfriz8GDhxpVZIkSRNkmJP6%0Aj0jyTeAxbdbrquqKkVYlSZI0QYbpIaOqLgeOGXEtkiRJE8lnWUqSJPXMQCZJktSzOQNZkkVJzhlX%0AMZIkSZNozkBWVbcD5ybZYkz1SJIkTZxhTuq/P3BmkpOBm6ZmVtWuI6tKkiRpggwTyN448iokSZIm%0A2DD3ITsxyUOArarq60nuCywafWmSJEmTYZiHi/8lcDTw/9qsTYHPj7AmSZKkiTLMbS/2B54A/AKg%0Aqn4CPGiURUmSJE2SYQLZLVX166kXSdYAar6Nknw0yVVJzhiY95Yklyb5Yft65sCy1yc5L8m5Sf5k%0ARQ9EkiTpnmqYQHZikoOAtZM8Dfhv4ItDbHcYsPMM8w+uqu3a15cBkmwN7A5s07b5YBLPU5MkSRNh%0AmEB2ILAcOB34K+DLwBvm26iqvgX8fMg6dgOOrKpbquoC4DzgsUNuK0mSdI82zFWWdyQ5HDiJbqjy%0A3Kqad8hyDi9PsiewDDigqq6lu1DgewPrXNLm/ZYk+wH7AWyxhferlSRJ93zDXGX5LOCnwPuADwDn%0AJXnGSu7vQ8CWwHbA5cC7V7SBqjqkqpZW1dLFixevZBmSJEkLxzA3hn03sGNVnQeQZEvgS8BXVnRn%0AVXXl1HSS/wSObS8vBTYfWHWzNk+SJGm1N8w5ZDdMhbHmfOCGldlZko0HXj4XmLoC8xhg9yT3SfJQ%0AYCvg5JXZhyRJ0j3NrD1kSZ7XJpcl+TJwFN05ZC8ATpmv4SRHAE8GNkxyCfBm4MlJtmvtXEh3kQBV%0AdWaSo4CzgNuA/duDzSVJklZ7cw1ZPntg+krgSW16ObD2fA1X1YtnmH3oHOu/A3jHfO1KkiStbmYN%0AZFW1zzgLkSRJmlTzntTfzul6BbBkcP2q2nV0ZUmSJE2OYa6y/DzdUOMXgTtGWo0kSdIEGiaQ/aqq%0A3jfySiRJkibUMIHsvUneDBwH3DI1s6pOHVlVkiRJE2SYQPZ7wB7ATtw1ZFnttSRJku6mYQLZC4CH%0AVdWvR12MJEnSJBrmTv1nABuMuA5JkqSJNUwP2QbAOUlO4TfPIfO2F5IkSavAMIHszSOvQpIkaYLN%0AG8iq6sRxFCJJkjSphrlT/w10V1UC3BtYE7ipqtYbZWGSJEmTYpgesnWnppME2A3YYZRFSZIkTZJh%0ArrK8U3U+D/zJaMqRJEmaPMMMWT5v4OW9gKXAr0ZWkSRJ0oQZ5irLZw9M3wZcSDdsKUmSpFVgmHPI%0A9hlHIZIkSZNq1kCW5E1zbFdV9fYR1CNJkjRx5uohu2mGefcD9gUeCBjIJEmSVoFZA1lVvXtqOsm6%0AwKuAfYAjgXfPtp0kSZJWzJznkCV5APBa4CXA4cD2VXXtOAqTJEmaFHOdQ/Yu4HnAIcDvVdWNY6tK%0AkiRpgsx1Y9gDgE2ANwCXJflF+7ohyS/GU54kSdLqb65zyFboLv6SJElaOYYuSZKknhnIJEmSemYg%0AkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFM%0AkiSpZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSerZyAJZko8muSrJ%0AGQPzHpDka0l+0r7fv81PkvclOS/JaUm2H1VdkiRJC80oe8gOA3aeNu9A4Piq2go4vr0GeAawVfva%0AD/jQCOuSJElaUEYWyKrqW8DPp83eDTi8TR8OPGdg/seq8z1ggyQbj6o2SZKkhWTc55BtVFWXt+kr%0AgI3a9KbAxQPrXdLm/ZYk+yVZlmTZ8uXLR1epJEnSmPR2Un9VFVArsd0hVbW0qpYuXrx4BJVJkiSN%0A17gD2ZVTQ5Ht+1Vt/qXA5gPrbdbmSZIkrfbGHciOAfZq03sBXxiYv2e72nIH4PqBoU1JkqTV2hqj%0AajjJEcCTgQ2TXAK8GXgncFSSfYGLgBe21b8MPBM4D/glsM+o6pIkSVpoRhbIqurFsyx6ygzrFrD/%0AqGqRJElayLxTvyRJUs8MZJIkST0zkEmSJPXMQCZJktQzA5kkSVLPDGSSJEk9M5BJkiT1zEAmSZLU%0AMwOZJElSzwxkkiRJPTOQSZIk9cxAJkmS1DMDmSRJUs8MZJIkST0zkEmSJPXMQCZJktQzA5kkSVLP%0ADGSSJEk9M5BJkiT1zEAmSZLUMwOZJElSzwxkkiRJPTOQSZIk9cxAJkmS1DMDmSRJUs8MZJIkST0z%0AkEmSJPXMQCZJktQzA5kkSVLPDGSSJEk9M5BJkiT1zEAmSZLUMwOZJElSzwxkkiRJPTOQSZIk9cxA%0AJkmS1DMDmSRJUs8MZJIkST0zkEmSJPXMQCZJktQzA5kkSVLPDGSSJEk9M5BJkiT1bI0+dprkQuAG%0A4HbgtqpamuQBwKeBJcCFwAur6to+6pMkSRqnPnvIdqyq7apqaXt9IHB8VW0FHN9eS5IkrfYW0pDl%0AbsDhbfpw4Dn9lSJJkjQ+fQWyAo5L8v0k+7V5G1XV5W36CmCjmTZMsl+SZUmWLV++fBy1SpIkjVQv%0A55ABT6yqS5M8CPhaknMGF1ZVJamZNqyqQ4BDAJYuXTrjOpIkSfckvfSQVdWl7ftVwOeAxwJXJtkY%0AoH2/qo/aJEmSxm3sgSzJ/ZKsOzUNPB04AzgG2KutthfwhXHXJkmS1Ic+hiw3Aj6XZGr/n6qq/0ly%0ACnBUkn2Bi4AX9lCbJEnS2I09kFXV+cCjZph/DfCUcdcjSZLUt4V02wtJkqSJZCCTJEnqmYFMkiSp%0AZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSe%0AGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnIJEmSemYgkyRJ6pmBTJIkqWcGMkmSpJ4ZyCRJknpm%0AIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnIJEmSemYgkyRJ6pmB%0ATJIkqWcGMkmSpJ4ZyCRJknpmIJMkSeqZgUySJKlnBjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYy%0ASZKknhnIJEmSemYgkybcxpttQZJ73NfGm23R91snSavMGn0XIKlfV1x6MQ953bF9l7HCLvqXXfou%0AYaVsvNkWXHHpxX2XscIevOnmXH7Jz/ouQ1ptLbhAlmRn4L3AIuAjVfXOnkuSpFXGACxpJgsqkCVZ%0ABPwH8DTgEuCUJMdU1Vn9ViZpwVm0Jkn6rkKSVokFFciAxwLnVdX5AEmOBHYDDGQT5J46pLPo3mtx%0A+69/1XcZk+P2W+1p0rzuqT9P4J47THxPfc/7fr9TVb3tfLokzwd2rqq/aK/3AB5XVS8fWGc/YL/2%0A8pHAuWMobUPg6jHsZyHy2CfXJB//JB87TPbxe+yTaxzH/5CqWjzTgoXWQzavqjoEOGSc+0yyrKqW%0AjnOfC4XHPpnHDpN9/JN87DDZx++xT+axQ//Hv9Bue3EpsPnA683aPEmSpNXWQgtkpwBbJXloknsD%0AuwPH9FyTJEnSSC2oIcuqui3Jy4Gv0t324qNVdWbPZcGYh0gXGI99ck3y8U/yscNkH7/HPrl6Pf4F%0AdVK/JEnSJFpoQ5aSJEkTx0AmSZLUMwPZHJLsnOTcJOclObDvesYpyeZJTkhyVpIzk7yq75rGLcmi%0AJD9Ics+7++jdkGSDJEcnOSfJ2Uke33dN45TkNe3f/BlJjkiyVt81jUqSjya5KskZA/MekORrSX7S%0Avt+/zxpHaZbjf1f7t39aks8l2aDHEkdmpmMfWHZAkkqyYR+1jcNsx5/kFe3zPzPJv46zJgPZLAYe%0A4/QMYGvgxUm27reqsboNOKCqtgZ2APafsOMHeBVwdt9F9OC9wP9U1e8Aj2KC3oMkmwKvBJZW1bZ0%0AFxft3m9VI3UYsPO0eQcCx1fVVsDx7fXq6jB++/i/BmxbVb8P/Bh4/biLGpPD+O1jJ8nmwNOBe94j%0AAlbMYUw7/iQ70j0d6FFVtQ3wb+MsyEA2uzsf41RVvwamHuM0Earq8qo6tU3fQPdLedN+qxqfJJsB%0AzwI+0nct45RkfeCPgUMBqurXVXVdr0WN3xrA2knWAO4LXNZzPSNTVd8Cfj5t9m7A4W36cOA546xp%0AnGY6/qo6rqpuay+/R3c/zNXOLJ89wMHA3wOr9RV/sxz/y4B3VtUtbZ2rxlmTgWx2mwKDD+O6hAkK%0AJIOSLAEeDZzUcynj9O90P5Tu6LmOcXsosBz4rzZc+5Ek9+u7qHGpqkvp/ir+GXA5cH1VHddvVWO3%0AUVVd3qavADbqs5ie/Tnwlb6LGJckuwGXVtWP+q6lJ48A/ijJSUlOTPKYce7cQKY5JVkH+Azw6qr6%0ARd/1jEOSXYCrqur7fdfSgzWA7YEPVdWjgZtYvYesfkM7X2o3umC6CXC/JC/tt6r+VHdfpNW6p2Q2%0ASf6B7tSNT/ZdyzgkuS9wEPCmvmvp0RrAA+hO0/k74KgkGdfODWSzm/jHOCVZky6MfbKqPtt3PWP0%0ABGDXJBfSDVXvlOQT/ZY0NpcAl1TVVG/o0XQBbVI8FbigqpZX1a3AZ4E/7LmmcbsyycYA7ftYh20W%0AgiR7A7sAL6nJuVnnlnR/iPyo/ezbDDg1yYN7rWq8LgE+W52T6UZIxnZhg4FsdhP9GKf2V8GhwNlV%0A9Z6+6xmnqnp9VW1WVUvoPvdvVNVE9JJU1RXAxUke2WY9BTirx5LG7WfADknu2/4PPIUJuqihOQbY%0Aq03vBXyhx1rGLsnOdKcr7FpVv+y7nnGpqtOr6kFVtaT97LsE2L79TJgUnwd2BEjyCODewNXj2rmB%0AbBbtpM6pxzidDRy1QB7jNC5PAPag6x36Yft6Zt9FaSxeAXwyyWnAdsA/9VvO+LSewaOBU4HT6X5G%0ArraPk0lyBPB/wCOTXJJkX+CdwNOS/ISux/CdfdY4SrMc/weAdYGvtZ97H+61yBGZ5dgnxizH/1Hg%0AYe1WGEcCe42zh9RHJ0mSJPXMHjJJkqSeGcgkSZJ6ZiCTJEnqmYFMkiSpZwYySZKknhnIJM0qyT8k%0AOTPJae0WAI+bZ/3Dkjz/buzvvkmuSbLetPmfT/KiWbZZ0i5TH5ska7dHqyxK8uQkx05bPu/7kGTX%0AJAe26eck2XqI/a6Z5NQ2fXv7TM5M8qMkByRZqZ/pSX4vyWErs62kVcNAJmlGSR5Pd7fy7avq9+nu%0ASXXx3FvdPe1GnF8FnjtQx/rAE4EvjnLfK+jP6e7offvKNlBVx1TV1D2+ngPMG8jo3ofvtumbq2q7%0AqtoGeBrwDODNK1nL6cBmSbZYme0l3X0GMkmz2Ri4uqpuAaiqq6vqMoAkb0pySpIzkhwy0/PekvxB%0A60X6fpKvDjyO55VJzmq9bkfOsN8j6J6QMOW5dCHt5iTvavs8faYesyR7J/nAwOtjkzy5Td/Ytj8z%0AydeTPDbJN5Ocn2TXts6its4prb6/muW9eQlD3sE+yYVJ3prk1Fb37wzWmuQPgV2Bd7Uery3neI92%0AZoaHXVfVVcB+wMvTWZLk222fp7Z9kORjSZ4zUNsn0z1QGrrAu/v0tiWNh4FM0myOAzZP8uMkH0zy%0ApIFlH6iqx1TVtsDadD1pd0r3HNT3A8+vqj+guwP2O9riA4FHt163v55hv18Ftk/ywPZ6d7qQ9jy6%0AJwc8iq637l1TIW9I96N7DNY2wA3AP9L1LD0XeFtbZ1/g+qp6DPAY4C+TPHTasd0beFhVXbgC+766%0AqrYHPgT87eCCqvpfuscV/V3r8fops79HOwLfnGkHVXU+sAh4EN3zJ5/W9vki4H1ttUOBvdtxrE/3%0AnM4vtWXLgD9agWOStAoZyCTNqKpuBP6ArudlOfDpdA9dBtgxyUlJTgd2AraZtvkjgW1pj58B3kD3%0AsGKA0+gezfRS4LYZ9vtruoDy/CQbAo+mC2lPBI6oqtur6krgRLrQNKxfA//Tpk8HTmwPED8dWNLm%0APx3Ys9V8EvBAYKtp7WwIXDdY8iz7G5z/2fb9+wP7mstvvUdJNgV+PuTzFdcE/rN9Pv9NGw6tqhPp%0AntG7GHgx8Jn2mDjoQtwmQ7QtaQTW6LsASQtXO0fqm8A32y/3vdoQ2geBpVV1cZK3AGtN2zTAmVX1%0A+BmafRbwx8CzgX9I8nsDoWDKEcAbWztfqKpbZxgVnclt/OYfmoN13TrwXLo7gKmh2DuSTP0sDPCK%0AqvrqHPu4eVq71wD3n7bOA/jNhxLf0r7fznA/d3/rPaIbrpy1riQPa+1fRXcu2ZV0vYn3An41sOrH%0AgJfS9TzuMzB/Lbpjk9QDe8gkzSjJI5MM9g5tB1zEXWHk6iTrADNdTXgusLhdGDB1deA27SrAzavq%0ABOB1wPrAOjNs/026nqn96cIZwLeBF7XzvBbTBZaTp213IbBdknsl2Rx47AocMnSB52VtyJUkj0hy%0Av8EVqupaYFGSqffhJ8AmSX63bfMQuiD0wxXY7w10D7RmjvdoxvPH2jaLgQ/TDSVX2+byqroD2INu%0AKHPKYcCr27GcNTD/EcBYr1aVdBd7yCTNZh3g/Uk2oOt5Og/Yr6quS/KfdL+8rwBOmb5hVf063W0f%0A3tfOVVoD+Hfgx8An2rwA76uq62bY/o4kRwMvpBuaBPgc8HjgR3TDgX9fVVckWTKw6XeBC4CzgLOB%0AU1fwmD9CN6R4artQYTndFZDTHUc3hPr1qrqlDS3+VwtptwJ/UVXXr8B+j6QbYnwlXc/VoYPvEV1g%0Ae3hVnTOwzdptaHVNus/n48B72rIPAp9JsifdMO1NUxtV1ZVJzgY+P62GHbnrfDJJY5a7evAlScNI%0Asj3wmqraY0z7eyLw0qqa6SKIFW3rvnTnzW0/FRqT3Icu+D5xhuFjSWPgkKUkraCqOhU4IcmieVde%0ANfv7zioKY0+l6zl8/7QevC2AAw1jUn/sIZMkSeqZPWSSJEk9M5BJkiT1zEAmSZLUMwOZJElSzwxk%0AkiRJPfv/1+FIq/hFCHkAAAAASUVORK5CYII=%0A)

### Normality Test<a href="#Normality-Test" class="anchor-link">¶</a>

> Can assume that the distribution of sales follows a normal
> distribution?

In \[23\]:

    # Bar Chart
    ax = plt.gca()
    colors = {False:'green', True:'red'}
    # Remove Outliers
    df_plot = df_abc[df_abc['CV']<4].copy()
    df_plot.plot.scatter(figsize=(12, 8), x='TO%', y='CV', color=df_plot['Not_Normal'].map(colors), ax =ax, grid = True)
    # ABC
    # A, B and C
    ax.axvline(to_a , color="red", linestyle="-", linewidth = 1.0)
    ax.axvline(to_b , color="red", linestyle="-", linewidth = 1.0)
    # 20%, 50% of SKU Number
    ax.axhline(1 , color="blue", linestyle="--", linewidth = 1.0)
    plt.xlabel('Percentage of Turnover (%)')
    plt.xticks(rotation=90)
    plt.ylabel('Coefficient of Variation')
    plt.title('Distribution by Demand Variability')
    plt.show()

![](...%0AAAAASUVORK5CYII=%0A)

In \[ \]:

     

In \[ \]:

     

In \[ \]: